In [1]:
import pyodbc
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sb
from scipy.stats import ttest_rel
import calendar
import warnings
warnings.filterwarnings('ignore')
# Configure inline mode
%matplotlib inline

# from dataprep.eda import *
pd.set_option('display.max_columns', None)

conn_sps = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Specific_Pavement_Studies.accdb"
    r";")

conn_mon= pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Monitoring.accdb"
    r";")

conn_adm = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Administration.accdb"
    r";")

# conn_mnt = pyodbc.connect(
#     r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
#     r"DBQ=D:\Documents\LTPP-IRI\Maint_Rehab.accdb"
#     r";")

conn_consolidated = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Consolidated.accdb"
    r";")

conn_drainage = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Drainage.accdb"
    r";")

conn_fwd = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\FWD_Data_Without_Drop_Data.accdb"
    r";")

conn_aws = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Auto_Weather_Station.accdb"
    r";")

conn_aws = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Auto_Weather_Station.accdb"
    r";")


conn_trf = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Traffic.accdb"
    r";")

conn_tst = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    r"DBQ=D:\Documents\LTPP-IRI\Material_Test.accdb"
    r";")

In [3]:
data_infor = pd.read_sql("select * from EXPERIMENT_SECTION", conn_sps)
data_sps1_layer = pd.read_sql("select * from SPS1_LAYER", conn_sps)
data_sps1_layer_thickness = pd.read_sql("select * from SPS1_LAYER_THICKNESS", conn_sps)
data_iri = pd.read_sql("select * from MON_HSS_PROFILE_SECTION", conn_mon)
data_iri_link = pd.read_sql("select * from MON_HSS_VISIT_NO",conn_mon)
data_crack = pd.read_sql("select * from MON_DIS_AC_REV", conn_mon)
data_friction = pd.read_sql("select * from MON_FRICTION",conn_mon)
data_rutting = pd.read_sql("select * from MON_T_PROF_INDEX_SECTION",conn_mon)

data_drainage = pd.read_sql("select * from MON_DRAIN_PERM_CALC",conn_drainage)

data_col_description = pd.read_sql("select * from INFOPAVE_LTPPDD", conn_adm)
# data_mnt = pd.read_sql("select * from MNT_IMP", conn_mnt) #
# data_rhb = pd.read_sql("select * from RHB_IMP", conn_mnt) #
# data_sps_construction = pd.read_sql("select * from CONSTRUCTION_EVENTS", conn_sps)  #
data_construction_event = pd.read_sql("select * from CONSTRUCTION_EVENTS_EXP", conn_consolidated)



data_aws_link = pd.read_sql("select * from AWS_LINK", conn_aws)
data_aws_humidity_month = pd.read_sql("select * from AWS_HUMIDITY_MONTH", conn_aws)
data_aws_precipitation_month = pd.read_sql("select * from AWS_PRECIPITATION_MONTH", conn_aws)
data_aws_solar_month = pd.read_sql("select * from AWS_SOLAR_MONTH", conn_aws)
data_aws_temp_month = pd.read_sql("select * from AWS_TEMP_MONTH", conn_aws)
data_aws_wind_month = pd.read_sql("select * from AWS_WIND_MONTH", conn_aws)
# data_col_description.to_csv('data_col_description.csv',index = False)

## basic data & construction

In [4]:
data_infor_sps1 = data_infor[(data_infor['GPS_SPS'] == 'S') & (data_infor['EXPERIMENT_NO'] == '1')]
data_infor_sps1['DATE'] = data_infor_sps1['CN_ASSIGN_DATE'] 
data_infor_sps1 = data_infor_sps1[['STATE_CODE','SHRP_ID','CONSTRUCTION_NO','DATE','CN_ASSIGN_DATE','ASSIGN_DATE']]
data_infor_sps1

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,CN_ASSIGN_DATE,ASSIGN_DATE
116,19,0108,1,1992-01-01,1992-01-01,1992-01-01
117,19,0109,1,1992-01-01,1992-01-01,1992-01-01
118,19,0110,1,1992-01-01,1992-01-01,1992-01-01
119,19,0111,1,1992-01-01,1992-01-01,1992-01-01
120,19,0112,1,1992-01-01,1992-01-01,1992-01-01
...,...,...,...,...,...,...
7133,30,0113,6,2012-06-01,2012-06-01,1997-09-18
7136,32,0111,2,2009-05-01,2009-05-01,1993-01-01
7354,22,0100,1,1992-11-18,1992-11-18,1992-11-18
7355,22,0113,1,1992-11-18,1992-11-18,1992-11-18


In [5]:
data_construction_event = data_construction_event[['STATE_CODE','SHRP_ID','IMP_DATE','IMP_TYPE','CONSTRUCTION_NO']]
data_construction_event = data_construction_event.rename(columns={"IMP_DATE": "DATE"})
# data_construction_event.head(50)

In [6]:
data_infor_sps1_con = pd.merge(data_infor_sps1,data_construction_event,how='left',on=['STATE_CODE','SHRP_ID','CONSTRUCTION_NO','DATE'])
data_infor_sps1_con.loc[data_infor_sps1_con['IMP_TYPE'].isna(),'IMP_TYPE'] = '0' #
data_infor_sps1_con = data_infor_sps1_con.sort_values(by=['STATE_CODE','SHRP_ID','CONSTRUCTION_NO'])
data_infor_sps1_con

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,CN_ASSIGN_DATE,ASSIGN_DATE,IMP_TYPE
251,1,0100,1,1991-04-30,1991-04-30,1991-04-30,0
252,1,0101,1,1991-04-30,1991-04-30,1991-04-30,0
253,1,0102,1,1991-04-30,1991-04-30,1991-04-30,0
325,1,0102,2,2003-04-17,2003-04-17,1991-04-30,24
326,1,0102,2,2003-04-17,2003-04-17,1991-04-30,27
...,...,...,...,...,...,...,...
248,55,0120,1,1997-01-01,1997-01-01,1997-01-01,0
236,55,0121,1,1997-01-01,1997-01-01,1997-01-01,0
237,55,0122,1,1997-01-01,1997-01-01,1997-01-01,0
238,55,0123,1,1997-01-01,1997-01-01,1997-01-01,0


In [6]:
data_infor_sps1_con[(data_infor_sps1_con['STATE_CODE'] == 1)&(data_infor_sps1_con['SHRP_ID'] == '0102')]

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,CN_ASSIGN_DATE,ASSIGN_DATE,IMP_TYPE
253,1,0102,1,1991-04-30,1991-04-30,1991-04-30,0
325,1,0102,2,2003-04-17,2003-04-17,1991-04-30,24
326,1,0102,2,2003-04-17,2003-04-17,1991-04-30,27


In [7]:
data_infor_sps1_con_group = data_infor_sps1_con.groupby(['STATE_CODE','SHRP_ID'])
len(list(data_infor_sps1_con_group.groups.keys()))

264

In [7]:
sps_1_state_code_list = list(set(data_infor_sps1_con['STATE_CODE']))
sps_1_shrp_id_list = list(set(data_infor_sps1_con['SHRP_ID']))

## IRI

In [4]:
data_iri_link = data_iri_link[['VISIT_NO','STATE_CODE','SHRP_ID','CONSTRUCTION_NO','VISIT_DATE']]
data_iri = data_iri[['VISIT_NO','RUN_NUMBER','IRI_LEFT_WHEEL_PATH','IRI_RIGHT_WHEEL_PATH','MRI']]
data_iri_combined = pd.merge(data_iri,data_iri_link,how='left',on='VISIT_NO')

In [5]:
data_iri_combined

,VISIT_NO,RUN_NUMBER,IRI_LEFT_WHEEL_PATH,IRI_RIGHT_WHEEL_PATH,MRI,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,VISIT_DATE
0,84680405,2,1.002,0.917,0.959,84,6804,2,1993-08-18
1,54400304,5,1.449,1.631,1.540,54,4003,1,1992-12-03
2,54400302,4,1.945,1.555,1.750,54,4003,1,1990-09-17
3,89300110,4,2.340,3.082,2.711,89,3001,3,1999-05-20
4,54164006,1,0.771,0.828,0.799,54,1640,2,1994-10-12
...,...,...,...,...,...,...,...,...,...
149953,380261MD,5,2.861,2.961,2.911,38,0261,8,2019-06-19
149954,380261ME,1,2.986,2.970,2.978,38,0261,8,2019-06-19
149955,380261ME,2,3.051,2.942,2.997,38,0261,8,2019-06-19
149956,380261ME,3,3.029,2.971,3.000,38,0261,8,2019-06-19


In [341]:
# data_iri_combined[(data_iri_combined['STATE_CODE'] == 30)&(data_iri_combined['SHRP_ID'] == '0113')&(data_iri_combined['VISIT_NO'] == '300113SP')]

In [9]:
data_iri_combined_mean = data_iri_combined.groupby('VISIT_NO')[['IRI_LEFT_WHEEL_PATH','IRI_RIGHT_WHEEL_PATH','MRI']].mean()
data_iri_combined_mean.columns = ['MEAN_IRI_LEFT_WHEEL_PATH', 'MEAN_IRI_RIGHT_WHEEL_PATH', 'MEAN_MRI']
data_iri_combined_mean = data_iri_combined_mean.reset_index()
data_iri_combined_mean

,VISIT_NO,MEAN_IRI_LEFT_WHEEL_PATH,MEAN_IRI_RIGHT_WHEEL_PATH,MEAN_MRI
0,01010101,0.6572,0.6574,0.6572
1,01010102,0.6962,0.6384,0.6670
2,01010103,0.6508,0.7148,0.6828
3,01010104,0.6804,0.7132,0.6970
4,01010105,0.6836,0.6724,0.6780
...,...,...,...,...
30502,90B35106,1.9330,1.9216,1.9276
30503,90B35107,2.0872,1.7618,1.9246
30504,90B35108,2.9020,2.0886,2.4952
30505,90B35109,2.8630,2.2032,2.5332


In [10]:
data_iri_mean = pd.merge(data_iri_combined_mean,data_iri_link,how='left',on='VISIT_NO')
data_iri_mean = data_iri_mean.rename(columns={"VISIT_NO": "IRI_VISIT_NO","VISIT_DATE": "IRI_VISIT_DATE"})
data_iri_mean['DATE'] = data_iri_mean['IRI_VISIT_DATE']
data_iri_mean

,IRI_VISIT_NO,MEAN_IRI_LEFT_WHEEL_PATH,MEAN_IRI_RIGHT_WHEEL_PATH,MEAN_MRI,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,IRI_VISIT_DATE,DATE
0,01010101,0.6572,0.6574,0.6572,1,0101,1,1995-10-30,1995-10-30
1,01010102,0.6962,0.6384,0.6670,1,0101,1,1996-01-10,1996-01-10
2,01010103,0.6508,0.7148,0.6828,1,0101,1,1996-04-08,1996-04-08
3,01010104,0.6804,0.7132,0.6970,1,0101,1,1996-08-12,1996-08-12
4,01010105,0.6836,0.6724,0.6780,1,0101,1,1996-10-11,1996-10-11
...,...,...,...,...,...,...,...,...,...
30502,90B35106,1.9330,1.9216,1.9276,90,B351,2,1995-04-26,1995-04-26
30503,90B35107,2.0872,1.7618,1.9246,90,B351,2,1995-06-24,1995-06-24
30504,90B35108,2.9020,2.0886,2.4952,90,B351,3,1997-04-29,1997-04-29
30505,90B35109,2.8630,2.2032,2.5332,90,B351,3,1998-08-28,1998-08-28


In [11]:
df = pd.concat([data_infor_sps1_con,data_iri_mean],sort=False)
df = df.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])
df.loc[df['IRI_VISIT_NO'].isna(),'EVENT'] = 'CN' 
df.loc[df['CN_ASSIGN_DATE'].isna(),'EVENT'] = 'IRI' 
df

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,CN_ASSIGN_DATE,ASSIGN_DATE,IMP_TYPE,IRI_VISIT_NO,MEAN_IRI_LEFT_WHEEL_PATH,MEAN_IRI_RIGHT_WHEEL_PATH,MEAN_MRI,IRI_VISIT_DATE,EVENT
251,1,0100,1,1991-04-30,1991-04-30,1991-04-30,0,NaN,NaN,NaN,NaN,NaT,CN
252,1,0101,1,1991-04-30,1991-04-30,1991-04-30,0,NaN,NaN,NaN,NaN,NaT,CN
0,1,0101,1,1995-10-30,NaT,NaT,NaN,01010101,0.6572,0.6574,0.6572,1995-10-30,IRI
1,1,0101,1,1996-01-10,NaT,NaT,NaN,01010102,0.6962,0.6384,0.6670,1996-01-10,IRI
2,1,0101,1,1996-04-08,NaT,NaT,NaN,01010103,0.6508,0.7148,0.6828,1996-04-08,IRI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30502,90,B351,2,1995-04-26,NaT,NaT,NaN,90B35106,1.9330,1.9216,1.9276,1995-04-26,IRI
30503,90,B351,2,1995-06-24,NaT,NaT,NaN,90B35107,2.0872,1.7618,1.9246,1995-06-24,IRI
30504,90,B351,3,1997-04-29,NaT,NaT,NaN,90B35108,2.9020,2.0886,2.4952,1997-04-29,IRI
30505,90,B351,3,1998-08-28,NaT,NaT,NaN,90B35109,2.8630,2.2032,2.5332,1998-08-28,IRI


## distress

In [12]:
data_crack = pd.read_sql("select * from MON_DIS_AC_REV", conn_mon)
data_crack = data_crack.rename(columns={"SURVEY_DATE": "DISTRESS_SURVEY_DATE"})
data_crack['DATE'] = data_crack['DISTRESS_SURVEY_DATE']

In [14]:
data_crack['GATOR_CRACK'] = data_crack[['GATOR_CRACK_A_L', 'GATOR_CRACK_A_M', 'GATOR_CRACK_A_H']].sum(axis=1)

data_crack['BLOCK_CRACK'] = data_crack[['BLK_CRACK_A_L', 'BLK_CRACK_A_M', 'BLK_CRACK_A_H']].sum(axis=1)

data_crack['EDGE_CRACK'] = data_crack[['EDGE_CRACK_L_L','EDGE_CRACK_L_M', 'EDGE_CRACK_L_H']].sum(axis=1)

data_crack['LONGITUDINAL_CRACK_WP'] = data_crack[['LONG_CRACK_WP_L_L','LONG_CRACK_WP_L_M', 'LONG_CRACK_WP_L_H',\
                                               'LONG_CRACK_WP_SEAL_L_L','LONG_CRACK_WP_SEAL_L_M', 'LONG_CRACK_WP_SEAL_L_H']].sum(axis=1)

data_crack['LONGITUDINAL_CRACK_NWP'] = data_crack[['LONG_CRACK_NWP_L_L', 'LONG_CRACK_NWP_L_M', 'LONG_CRACK_NWP_L_H',\
                                                   'LONG_CRACK_NWP_SEAL_L_L', 'LONG_CRACK_NWP_SEAL_L_M','LONG_CRACK_NWP_SEAL_L_H']].sum(axis=1)

data_crack['TRANS_CRACK_NUMBER'] = data_crack[['TRANS_CRACK_NO_L', 'TRANS_CRACK_NO_M','TRANS_CRACK_NO_H']].sum(axis=1)

data_crack['TRANS_CRACK_LENGTH'] = data_crack[['TRANS_CRACK_L_L', 'TRANS_CRACK_L_M','TRANS_CRACK_L_H', \
                                            'TRANS_CRACK_SEAL_L_L', 'TRANS_CRACK_SEAL_L_M','TRANS_CRACK_SEAL_L_H']].sum(axis=1)

data_crack['PATCH_NUMBER'] = data_crack[['PATCH_NO_L', 'PATCH_NO_M', 'PATCH_NO_H']].sum(axis=1)

data_crack['PATCH_AREA'] = data_crack[['PATCH_A_L', 'PATCH_A_M', 'PATCH_A_H']].sum(axis=1)

data_crack['POTHOLES_NUMBER'] = data_crack[['POTHOLES_NO_L', 'POTHOLES_NO_M','POTHOLES_NO_H']].sum(axis=1)

data_crack['POTHOLES_AREA'] = data_crack[['POTHOLES_A_L', 'POTHOLES_A_M', 'POTHOLES_A_H']].sum(axis=1)

# 'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A', 'RAVELING','PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED','TRANS_CRACK_L_GT183'

In [15]:
data_crack = data_crack[['STATE_CODE', 'SHRP_ID', 'DISTRESS_SURVEY_DATE','DATE', 'CONSTRUCTION_NO',\
                          'GATOR_CRACK', 'BLOCK_CRACK', 'EDGE_CRACK','LONGITUDINAL_CRACK_WP', 'LONGITUDINAL_CRACK_NWP', \
                          'TRANS_CRACK_NUMBER','TRANS_CRACK_LENGTH', 'PATCH_NUMBER', 'PATCH_AREA', 'POTHOLES_NUMBER','POTHOLES_AREA',\
                         'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A', 'RAVELING','PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED','TRANS_CRACK_L_GT183']]

In [16]:
data_crack = data_crack.fillna(0)

In [17]:
data_crack.columns

Index(['STATE_CODE', 'SHRP_ID', 'DISTRESS_SURVEY_DATE', 'DATE',
       'CONSTRUCTION_NO', 'GATOR_CRACK', 'BLOCK_CRACK', 'EDGE_CRACK',
       'LONGITUDINAL_CRACK_WP', 'LONGITUDINAL_CRACK_NWP', 'TRANS_CRACK_NUMBER',
       'TRANS_CRACK_LENGTH', 'PATCH_NUMBER', 'PATCH_AREA', 'POTHOLES_NUMBER',
       'POTHOLES_AREA', 'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A',
       'RAVELING', 'PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED',
       'TRANS_CRACK_L_GT183'],
      dtype='object')

In [18]:
df_1 = pd.concat([df,data_crack],sort=False)
df_1.loc[-df_1['DISTRESS_SURVEY_DATE'].isna(),'EVENT'] = 'DISTRESS' 
df_1 = df_1.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])

In [19]:
df_1.columns

Index(['STATE_CODE', 'SHRP_ID', 'CONSTRUCTION_NO', 'DATE', 'CN_ASSIGN_DATE',
       'ASSIGN_DATE', 'IMP_TYPE', 'IRI_VISIT_NO', 'MEAN_IRI_LEFT_WHEEL_PATH',
       'MEAN_IRI_RIGHT_WHEEL_PATH', 'MEAN_MRI', 'IRI_VISIT_DATE', 'EVENT',
       'DISTRESS_SURVEY_DATE', 'GATOR_CRACK', 'BLOCK_CRACK', 'EDGE_CRACK',
       'LONGITUDINAL_CRACK_WP', 'LONGITUDINAL_CRACK_NWP', 'TRANS_CRACK_NUMBER',
       'TRANS_CRACK_LENGTH', 'PATCH_NUMBER', 'PATCH_AREA', 'POTHOLES_NUMBER',
       'POTHOLES_AREA', 'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A',
       'RAVELING', 'PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED',
       'TRANS_CRACK_L_GT183'],
      dtype='object')

## rutting

In [23]:
data_rutting = pd.read_sql("select * from MON_T_PROF_INDEX_SECTION",conn_mon)
data_rutting


,VISIT_NO,RUN_NUMBER,STATE_CODE,SHRP_ID,SURVEY_DATE,CONSTRUCTION_NO,RECORD_STATUS,T_PROF_DEVICE_CODE,NO_PROFILES,NEGATIVE_AREA_MEAN,NEGATIVE_AREA_STD,NEGATIVE_AREA_MIN,NEGATIVE_AREA_MAX,POSITIVE_AREA_MEAN,POSITIVE_AREA_STD,POSITIVE_AREA_MIN,POSITIVE_AREA_MAX,FILL_AREA_MEAN,FILL_AREA_STD,FILL_AREA_MIN,FILL_AREA_MAX,LLH_DEPTH_1_8_MEAN,LLH_DEPTH_1_8_STD,LLH_DEPTH_1_8_MIN,LLH_DEPTH_1_8_MAX,RLH_DEPTH_1_8_MEAN,RLH_DEPTH_1_8_STD,RLH_DEPTH_1_8_MIN,RLH_DEPTH_1_8_MAX,MAX_MEAN_DEPTH_1_8,LLH_WIDTH_1_8_MEAN,LLH_WIDTH_1_8_STD,LLH_WIDTH_1_8_MIN,LLH_WIDTH_1_8_MAX,LLH_OFFSET_1_8_MEAN,LLH_OFFSET_1_8_STD,LLH_OFFSET_1_8_MIN,LLH_OFFSET_1_8_MAX,RLH_WIDTH_1_8_MEAN,RLH_WIDTH_1_8_STD,RLH_WIDTH_1_8_MIN,RLH_WIDTH_1_8_MAX,RLH_OFFSET_1_8_MEAN,RLH_OFFSET_1_8_STD,RLH_OFFSET_1_8_MIN,RLH_OFFSET_1_8_MAX,LLH_DEPTH_WIRE_REF_MEAN,LLH_DEPTH_WIRE_REF_STD,LLH_DEPTH_WIRE_REF_MIN,LLH_DEPTH_WIRE_REF_MAX,RLH_DEPTH_WIRE_REF_MEAN,RLH_DEPTH_WIRE_REF_STD,RLH_DEPTH_WIRE_REF_MIN,RLH_DEPTH_WIRE_REF_MAX,MAX_MEAN_DEPTH_WIRE_REF,LLH_WIDTH_WIRE_REF_MEAN,LLH_WIDTH_WIRE_REF_STD,LLH_WIDTH_WIRE_REF_MIN,LLH_WIDTH_WIRE_REF_MAX,LLH_OFFSET_WIRE_REF_MEAN,LLH_OFFSET_WIRE_REF_STD,LLH_OFFSET_WIRE_REF_MIN,LLH_OFFSET_WIRE_REF_MAX,RLH_WIDTH_WIRE_REF_MEAN,RLH_WIDTH_WIRE_REF_STD,RLH_WIDTH_WIRE_REF_MIN,RLH_WIDTH_WIRE_REF_MAX,RLH_OFFSET_WIRE_REF_MEAN,RLH_OFFSET_WIRE_REF_STD,RLH_OFFSET_WIRE_REF_MIN,RLH_OFFSET_WIRE_REF_MAX,DATA_PROCESS_EXTRACT_DATE
0,04100620,1,4,1006,2014-12-02,5,E,D,11,-7162,1685,-4964,-10057,2620,1070,1029,4808,11179,1397,9142,13561,9,1,7,11,5,1,4,7,9,1524,0,1524,1524,2715,92,2438,2743,1441,142,1219,1524,638,92,610,914,9,1,7,11,6,1,4,7,9,1552,92,1524,1829,2715,92,2438,2743,1579,267,1219,1829,638,92,610,914,2020-03-11 21:38:34
1,04102215,1,4,1022,2008-03-12,3,E,D,11,-4335,2298,-1220,-9728,6940,2936,2414,12498,8658,2009,6705,13715,9,2,7,12,3,1,2,5,9,1524,0,1524,1524,2715,92,2438,2743,1247,288,914,1524,914,0,914,914,9,2,7,12,3,1,2,5,9,1552,92,1524,1829,2715,92,2438,2743,1247,288,914,1524,914,0,914,914,2020-03-11 21:38:34
2,04103409,1,4,1034,2001-05-08,4,E,D,11,-16211,2916,-11760,-20720,117,270,0,800,16855,2583,12492,20720,6,1,5,8,11,2,9,14,11,1524,0,1524,1524,2604,159,2438,2743,1524,0,1524,1524,776,159,610,914,7,1,6,10,13,2,10,16,13,3159,550,1524,3353,2577,159,2438,2743,3159,550,1524,3353,776,159,610,914,2020-03-11 21:38:34
3,04101710,1,4,1017,1996-09-26,2,E,D,11,-18794,3001,-15033,-23468,1060,968,0,3424,23041,2463,19663,28042,11,2,8,13,11,3,7,16,11,1524,0,1524,1524,2854,154,2743,3048,1524,0,1524,1524,914,136,610,1219,11,2,9,14,13,3,9,17,13,2051,808,1524,3658,2826,142,2743,3048,2162,752,1524,3658,886,92,610,914,2020-03-11 21:38:34
4,04101502,1,4,1015,1990-07-02,1,E,P,9,-27253,5483,-19203,-35698,120,246,0,661,27964,4943,22443,35698,10,2,7,13,15,3,12,22,15,1631,145,1406,1788,2926,56,2840,3008,1705,35,1661,1753,785,71,651,893,10,2,7,14,18,4,13,25,18,2930,1086,1406,3660,2926,56,2840,3008,3143,767,2026,3660,785,71,651,893,2020-03-11 21:38:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21365,081053GR,1,8,1053,2018-10-24,2,E,L,499,-5500,1215,-2715,-10487,1763,779,290,5688,8780,937,6468,12483,7,1,5,10,4,1,3,7,7,1499,53,1362,1711,2508,70,2356,2730,1441,133,372,1663,757,93,131,978,7,1,5,10,4,1,3,7,7,1499,53,1362,1711,2508,70,2356,2730,1441,133,372,1663,757,93,131,978,2020-03-11 21:38:34
21366,08105313,1,8,1053,1996-05-10,1,E,P,10,-13987,3355,-9262,-19863,1624,1306,0,3858,19445,1200,17816,21561,10,2,5,13,12,2,9,14,12,1485,130,1282,1661,2795,160,2630,3158,1740,39,1654,1781,963,117,773,1110,10,2,6,14,12,1,10,14,12,2241,946,1330,3421,2778,151,2630,3158,2470,753,1654,3421,999,107,777,1131,2020-03-11 21:38:34
21367,08102909,1,8,1029,2001-08-21,3,E,D,11,-2399,6025,0,-20013,16255,11000,203,32615,8396,5133,3964,21639,5,1,3,8,5,2,2,9,5,1413,154,1219,1524,2771,2

In [24]:
data_rutting = pd.read_sql("select * from MON_T_PROF_INDEX_SECTION",conn_mon)
data_rutting['RUTTING']=data_rutting[['LLH_DEPTH_WIRE_REF_MAX','RLH_DEPTH_WIRE_REF_MAX']].mean(axis=1)
data_rutting = data_rutting[['VISIT_NO','STATE_CODE', 'SHRP_ID', 'SURVEY_DATE','CONSTRUCTION_NO', 'RUTTING']]
data_rutting = data_rutting.rename(columns={"VISIT_NO":"RUTTING_VISIT_NO","SURVEY_DATE": "RUTTING_SURVEY_DATE"})
data_rutting['DATE'] = data_rutting['RUTTING_SURVEY_DATE']

In [25]:
data_rutting.columns

Index(['RUTTING_VISIT_NO', 'STATE_CODE', 'SHRP_ID', 'RUTTING_SURVEY_DATE',
       'CONSTRUCTION_NO', 'RUTTING', 'DATE'],
      dtype='object')

In [26]:
data_rutting = data_rutting.fillna(0)

In [27]:
df_2 = pd.concat([df_1,data_rutting],sort=False)
df_2.loc[-df_2['RUTTING_SURVEY_DATE'].isna(),'EVENT'] = 'RUTTING' 
df_2 = df_2.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])

In [79]:
# df_2[(df_2['STATE_CODE'] == 1)&(df_2['SHRP_ID'] == '0102')]

## friction

In [28]:
data_friction = pd.read_sql("select * from MON_FRICTION",conn_mon)

In [29]:
data_friction['FRICTION']=data_friction[['FRICTION_NO_BEGIN', 'FRICTION_NO_END']].mean(axis=1)
data_friction = data_friction[['SHRP_ID','STATE_CODE', 'CONSTRUCTION_NO', 'FRICTION_DATE','FRICTION']]
data_friction['DATE'] = data_friction['FRICTION_DATE']


In [30]:
data_friction.columns

Index(['SHRP_ID', 'STATE_CODE', 'CONSTRUCTION_NO', 'FRICTION_DATE', 'FRICTION',
       'DATE'],
      dtype='object')

In [31]:
# data_friction[(data_friction['FRICTION'].isna())]

In [32]:
data_friction = data_friction.fillna(0)

In [33]:
df_3 = pd.concat([df_2,data_friction],sort=False)
df_3.loc[-df_3['FRICTION_DATE'].isna(),'EVENT'] = 'FRICTION' 
df_3 = df_3.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])

In [90]:
# df_3[(df_3['STATE_CODE'] == 1)&(df_3['SHRP_ID'] == '0102')]

## drainage

In [34]:
data_drainage = pd.read_sql("select * from MON_DRAIN_PERM_CALC",conn_drainage)

In [35]:
data_drainage = data_drainage[['STATE_CODE', 'SHRP_ID', 'CONSTRUCTION_NO','TEST_DATE','PVMT_THICK_DB', 'PATB_THICK_DB','HYDRAULIC_CONDUCTIVITY']]
data_drainage = data_drainage.rename(columns={"TEST_DATE": "DRAINAGE_TEST_DATE"})
data_drainage['DATE'] = data_drainage['DRAINAGE_TEST_DATE']


In [36]:
data_drainage.columns

Index(['STATE_CODE', 'SHRP_ID', 'CONSTRUCTION_NO', 'DRAINAGE_TEST_DATE',
       'PVMT_THICK_DB', 'PATB_THICK_DB', 'HYDRAULIC_CONDUCTIVITY', 'DATE'],
      dtype='object')

In [37]:
data_drainage['HYDRAULIC_CONDUCTIVITY'] = data_drainage['HYDRAULIC_CONDUCTIVITY'].fillna(data_drainage['HYDRAULIC_CONDUCTIVITY'].mean())

In [38]:
df_4 = pd.concat([df_3,data_drainage],sort=False)
df_4.loc[-df_4['DRAINAGE_TEST_DATE'].isna(),'EVENT'] = 'DRAINAGE' 
df_4 = df_4.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])

In [981]:
# df_4[(df_4['EVENT'] == 'DISTRESS')&(df_4['BLEEDING'].isna())]

In [39]:
data_drainage
data_drainage_group = data_drainage.groupby(['STATE_CODE','SHRP_ID'])
len(list(data_drainage_group.groups.keys()))

143

## fwd

In [40]:
data_fwd = pd.read_csv('modulus.csv',index_col=None)
data_fwd_link = pd.read_csv('modulus_link.csv',index_col=None)
data_fwd = data_fwd[['STATE_CODE', 'SHRP_ID', 'FWD_PASS', 'BC_LAYER_NO', 'AVG_MODULUS']]
data_fwd_link = data_fwd_link[['STATE_CODE', 'SHRP_ID', 'FWD_PASS', 'TEST_DATE', 'CONSTRUCTION_NO']]

data_fwd

,STATE_CODE,SHRP_ID,FWD_PASS,BC_LAYER_NO,AVG_MODULUS
0,27,1016,31,1,1212.1
1,27,0509,2,1,387.4
2,27,6251,196,1,868.6
3,83,0502,45,1,1047.5
4,24,0563,14,1,357.5
...,...,...,...,...,...
527132,29,0603,44,5,500.0
527133,48,A504,45,5,500.0
527134,48,1122,1105,5,61.4
527135,1,0661,30,5,59.1


In [182]:
# data_fwd_link.columns

In [41]:
data_fwd_merge = pd.merge(data_fwd,data_fwd_link,how='left',on=['STATE_CODE', 'SHRP_ID', 'FWD_PASS'])
data_fwd_merge = data_fwd_merge.sort_values(by=['STATE_CODE','SHRP_ID','FWD_PASS'])

In [42]:
data_fwd_merge_mean=data_fwd_merge.groupby(['STATE_CODE','SHRP_ID','BC_LAYER_NO','TEST_DATE','CONSTRUCTION_NO'])[['AVG_MODULUS']].mean()
data_fwd_merge_mean = data_fwd_merge_mean.reset_index()
data_fwd_merge_mean = data_fwd_merge_mean.sort_values(by=['STATE_CODE','SHRP_ID','TEST_DATE'])
data_fwd_merge_mean = data_fwd_merge_mean.reset_index()
del data_fwd_merge_mean['index']
# data_modulus = data_fwd_merge_mean


In [43]:
data_fwd_merge_mean = data_fwd_merge_mean.rename(columns={"TEST_DATE": "MODULUS_TEST_DATE"})
data_fwd_merge_mean['DATE'] = data_fwd_merge_mean['MODULUS_TEST_DATE']
data_fwd_merge_mean.head(10)

,STATE_CODE,SHRP_ID,BC_LAYER_NO,MODULUS_TEST_DATE,CONSTRUCTION_NO,AVG_MODULUS,DATE
0,1,0101,1,1993-03-11 00:00:00.0000000,1.0,422.062500,1993-03-11 00:00:00.0000000
1,1,0101,2,1993-03-11 00:00:00.0000000,1.0,20.775000,1993-03-11 00:00:00.0000000
2,1,0101,3,1993-03-11 00:00:00.0000000,1.0,38.125000,1993-03-11 00:00:00.0000000
3,1,0101,4,1993-03-11 00:00:00.0000000,1.0,26.412500,1993-03-11 00:00:00.0000000
4,1,0101,1,1995-06-21 00:00:00.0000000,1.0,280.362500,1995-06-21 00:00:00.0000000
5,1,0101,2,1995-06-21 00:00:00.0000000,1.0,26.750000,1995-06-21 00:00:00.0000000
6,1,0101,3,1995-06-21 00:00:00.0000000,1.0,31.575000,1995-06-21 00:00:00.0000000
7,1,0101,4,1995-06-21 00:00:00.0000000,1.0,27.325000,1995-06-21 00:00:00.0000000
8,1,0101,1,1995-07-26 00:00:00.0000000,1.0,246.895833,1995-07-26 00:00:00.0000000
9,1,0101,2,1995-07-26 00:00:00.0000000,1.0,38.070833,1995-07-26 00:00:00.0000000


In [44]:
data_modulus = pd.DataFrame([])
data_modulus['STATE_CODE'] = None
data_modulus['SHRP_ID'] = None
data_modulus['MODULUS_TEST_DATE'] = None
data_modulus['CONSTRUCTION_NO'] = None
data_modulus['DATE'] = None
data_modulus['BC_LAYER_NO_1_MODULUS'] = None
data_modulus['BC_LAYER_NO_2_MODULUS'] = None
data_modulus['BC_LAYER_NO_3_MODULUS'] = None
data_modulus['BC_LAYER_NO_4_MODULUS'] = None
data_modulus['BC_LAYER_NO_5_MODULUS'] = None


data_fwd_merge_mean_group = data_fwd_merge_mean.groupby(['STATE_CODE','SHRP_ID','MODULUS_TEST_DATE','CONSTRUCTION_NO'])
# j=0
for j in range(len(list(data_fwd_merge_mean_group.groups.keys()))):
    x = list(data_fwd_merge_mean_group.groups.keys())[j]  #key
    y = data_fwd_merge_mean.loc[data_fwd_merge_mean_group.groups[list(data_fwd_merge_mean_group.groups.keys())[j]],:]  #extracted dataframe

    data_modulus = data_modulus.append({'STATE_CODE':x[0],\
                                        'SHRP_ID':x[1],\
                                        'MODULUS_TEST_DATE':x[2],\
                                        'CONSTRUCTION_NO':x[3],\
                                        'DATE':x[2],\
                                        'BC_LAYER_NO_1_MODULUS':y.loc[y['BC_LAYER_NO']==1,'AVG_MODULUS'].item() if len(y.loc[y['BC_LAYER_NO']==1,'AVG_MODULUS'])==1 else 0,\
                                        'BC_LAYER_NO_2_MODULUS':y.loc[y['BC_LAYER_NO']==2,'AVG_MODULUS'].item() if len(y.loc[y['BC_LAYER_NO']==2,'AVG_MODULUS'])==1 else 0,\
                                        'BC_LAYER_NO_3_MODULUS':y.loc[y['BC_LAYER_NO']==3,'AVG_MODULUS'].item() if len(y.loc[y['BC_LAYER_NO']==3,'AVG_MODULUS'])==1 else 0,\
                                        'BC_LAYER_NO_4_MODULUS':y.loc[y['BC_LAYER_NO']==4,'AVG_MODULUS'].item() if len(y.loc[y['BC_LAYER_NO']==4,'AVG_MODULUS'])==1 else 0,\
                                        'BC_LAYER_NO_5_MODULUS':y.loc[y['BC_LAYER_NO']==5,'AVG_MODULUS'].item() if len(y.loc[y['BC_LAYER_NO']==5,'AVG_MODULUS']) == 1 else 0},\
                                        ignore_index=True)

    if j%1000==0: print(j)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [45]:
data_modulus.to_csv('data_modulus.csv',index = False)
# data_modulus = pd.read_csv('data_modulus.csv',index_col=None)


In [46]:
df_5 = pd.concat([df_4,data_modulus],sort=False)
df_5.loc[-df_5['MODULUS_TEST_DATE'].isna(),'EVENT'] = 'FWD' 
df_5 = df_5.sort_values(by=['STATE_CODE','SHRP_ID','DATE'])

## fix missing data

In [47]:
df_5_sps1 = df_5[(df_5['STATE_CODE'].isin(sps_1_state_code_list)) & (df_5['SHRP_ID'].isin(sps_1_shrp_id_list))]

In [48]:
df_5_sps1 = df_5_sps1.reset_index()
del df_5_sps1['index']

In [49]:
df_6 = pd.DataFrame([])

df_5_sps1_group = df_5_sps1.groupby(['STATE_CODE','SHRP_ID'])
# j=2
for j in range(len(list(df_5_sps1_group.groups.keys()))):
    x = list(df_5_sps1_group.groups.keys())[j]  #key
    y = df_5_sps1.loc[df_5_sps1_group.groups[list(df_5_sps1_group.groups.keys())[j]],:]  #extracted dataframe
    ASSIGN_DATE = y.loc[-y['ASSIGN_DATE'].isna(),'ASSIGN_DATE'].unique()[0]
    y['ASSIGN_DATE'] = ASSIGN_DATE
    y['DATE'] = pd.to_datetime(y['DATE']) 
    y['AGE'] = (y['DATE'] - y['ASSIGN_DATE'])/np.timedelta64(1, 'Y')
    
    # initial iri
    try:
        y_iri_only = y[y['EVENT'] == 'IRI']
        Y_IRI = y_iri_only['MEAN_MRI']
        X_IRI = pd.DataFrame({'Const':1,
                          'AGE':y_iri_only['AGE']})

        # Describe the linear model
        mod_iri = sm.OLS(Y_IRI,X_IRI)

        # Fit model
        fit_iri = mod_iri.fit()

        # Summarize model
        # fit_iri.summary()

        initial_iri = fit_iri.params['Const']

        y_iri_only['INITIAL_IRI'] = initial_iri
    except:
        y_iri_only['INITIAL_IRI'] = 0
    
    

    # DISTRESS
    try:
        y_distress_only = y[y['EVENT'] == 'DISTRESS']

        Y_GATOR_CRACK = y_distress_only['GATOR_CRACK']
        Y_BLOCK_CRACK = y_distress_only['BLOCK_CRACK']
        Y_EDGE_CRACK = y_distress_only['EDGE_CRACK']
        Y_LONGITUDINAL_CRACK_WP = y_distress_only['LONGITUDINAL_CRACK_WP']
        Y_LONGITUDINAL_CRACK_NWP = y_distress_only['LONGITUDINAL_CRACK_NWP']
        Y_TRANS_CRACK_NUMBER = y_distress_only['TRANS_CRACK_NUMBER']
        Y_TRANS_CRACK_LENGTH = y_distress_only['TRANS_CRACK_LENGTH']
        Y_PATCH_NUMBER = y_distress_only['PATCH_NUMBER']
        Y_PATCH_AREA = y_distress_only['PATCH_AREA']
        Y_POTHOLES_NUMBER = y_distress_only['POTHOLES_NUMBER']
        Y_POTHOLES_AREA = y_distress_only['POTHOLES_AREA']
        Y_SHOVING_NO = y_distress_only['SHOVING_NO']
        Y_SHOVING_A = y_distress_only['SHOVING_A']
        Y_BLEEDING = y_distress_only['BLEEDING']
        Y_POLISH_AGG_A = y_distress_only['POLISH_AGG_A']
        Y_RAVELING = y_distress_only['RAVELING']
        Y_PUMPING_NO = y_distress_only['PUMPING_NO']
        Y_PUMPING_L = y_distress_only['PUMPING_L']
        Y_WP_LENGTH_CRACKED = y_distress_only['WP_LENGTH_CRACKED']
        Y_TRANS_CRACK_L_GT183 = y_distress_only['TRANS_CRACK_L_GT183']




        X_DISTRESS = pd.DataFrame({'Const':1,
                          'AGE':y_distress_only['AGE']})

        # Describe the linear model
        mod_GATOR_CRACK = sm.OLS(Y_GATOR_CRACK,X_DISTRESS)
        mod_BLOCK_CRACK = sm.OLS(Y_BLOCK_CRACK,X_DISTRESS)
        mod_EDGE_CRACK = sm.OLS(Y_EDGE_CRACK,X_DISTRESS)
        mod_LONGITUDINAL_CRACK_WP = sm.OLS(Y_LONGITUDINAL_CRACK_WP,X_DISTRESS)
        mod_LONGITUDINAL_CRACK_NWP = sm.OLS(Y_LONGITUDINAL_CRACK_NWP,X_DISTRESS)
        mod_TRANS_CRACK_NUMBER = sm.OLS(Y_TRANS_CRACK_NUMBER,X_DISTRESS)
        mod_TRANS_CRACK_LENGTH = sm.OLS(Y_TRANS_CRACK_LENGTH,X_DISTRESS)
        mod_PATCH_NUMBER = sm.OLS(Y_PATCH_NUMBER,X_DISTRESS)
        mod_PATCH_AREA = sm.OLS(Y_PATCH_AREA,X_DISTRESS)
        mod_POTHOLES_NUMBER = sm.OLS(Y_POTHOLES_NUMBER,X_DISTRESS)
        mod_POTHOLES_AREA = sm.OLS(Y_POTHOLES_AREA,X_DISTRESS)
        mod_SHOVING_NO = sm.OLS(Y_SHOVING_NO,X_DISTRESS)
        mod_SHOVING_A = sm.OLS(Y_SHOVING_A,X_DISTRESS)
        mod_BLEEDING = sm.OLS(Y_BLEEDING,X_DISTRESS)
        mod_POLISH_AGG_A = sm.OLS(Y_POLISH_AGG_A,X_DISTRESS)
        mod_RAVELING = sm.OLS(Y_RAVELING,X_DISTRESS)
        mod_PUMPING_NO = sm.OLS(Y_PUMPING_NO,X_DISTRESS)
        mod_PUMPING_L = sm.OLS(Y_PUMPING_L,X_DISTRESS)
        mod_WP_LENGTH_CRACKED = sm.OLS(Y_WP_LENGTH_CRACKED,X_DISTRESS)
        mod_TRANS_CRACK_L_GT183 = sm.OLS(Y_TRANS_CRACK_L_GT183,X_DISTRESS)



        # Fit model
        fit_GATOR_CRACK = mod_GATOR_CRACK.fit()
        fit_BLOCK_CRACK = mod_BLOCK_CRACK.fit()
        fit_EDGE_CRACK = mod_EDGE_CRACK.fit()
        fit_LONGITUDINAL_CRACK_WP = mod_LONGITUDINAL_CRACK_WP.fit()
        fit_LONGITUDINAL_CRACK_NWP = mod_LONGITUDINAL_CRACK_NWP.fit()
        fit_TRANS_CRACK_NUMBER = mod_TRANS_CRACK_NUMBER.fit()
        fit_TRANS_CRACK_LENGTH = mod_TRANS_CRACK_LENGTH.fit()
        fit_PATCH_NUMBER = mod_PATCH_NUMBER.fit()
        fit_PATCH_AREA = mod_PATCH_AREA.fit()
        fit_POTHOLES_NUMBER = mod_POTHOLES_NUMBER.fit()
        fit_POTHOLES_AREA = mod_POTHOLES_AREA.fit()
        fit_SHOVING_NO = mod_SHOVING_NO.fit()
        fit_SHOVING_A = mod_SHOVING_A.fit()
        fit_BLEEDING = mod_BLEEDING.fit()
        fit_POLISH_AGG_A = mod_POLISH_AGG_A.fit()
        fit_RAVELING = mod_RAVELING.fit()
        fit_PUMPING_NO = mod_PUMPING_NO.fit()
        fit_PUMPING_L = mod_PUMPING_L.fit()
        fit_WP_LENGTH_CRACKED = mod_WP_LENGTH_CRACKED.fit()
        fit_TRANS_CRACK_L_GT183 = mod_TRANS_CRACK_L_GT183.fit()


        y_iri_only['GATOR_CRACK'] = fit_GATOR_CRACK.params['AGE'] * y_iri_only['AGE'] + fit_GATOR_CRACK.params['Const']
        y_iri_only['BLOCK_CRACK'] = fit_BLOCK_CRACK.params['AGE'] * y_iri_only['AGE'] + fit_BLOCK_CRACK.params['Const']
        y_iri_only['EDGE_CRACK'] = fit_EDGE_CRACK.params['AGE'] * y_iri_only['AGE'] + fit_EDGE_CRACK.params['Const']
        y_iri_only['LONGITUDINAL_CRACK_WP'] = fit_LONGITUDINAL_CRACK_WP.params['AGE'] * y_iri_only['AGE'] + fit_LONGITUDINAL_CRACK_WP.params['Const']
        y_iri_only['LONGITUDINAL_CRACK_NWP'] = fit_LONGITUDINAL_CRACK_NWP.params['AGE'] * y_iri_only['AGE'] + fit_LONGITUDINAL_CRACK_NWP.params['Const']
        y_iri_only['TRANS_CRACK_NUMBER'] = fit_TRANS_CRACK_NUMBER.params['AGE'] * y_iri_only['AGE'] + fit_TRANS_CRACK_NUMBER.params['Const']
        y_iri_only['TRANS_CRACK_LENGTH'] = fit_TRANS_CRACK_LENGTH.params['AGE'] * y_iri_only['AGE'] + fit_TRANS_CRACK_LENGTH.params['Const']
        y_iri_only['PATCH_NUMBER'] = fit_PATCH_NUMBER.params['AGE'] * y_iri_only['AGE'] + fit_PATCH_NUMBER.params['Const']
        y_iri_only['PATCH_AREA'] = fit_PATCH_AREA.params['AGE'] * y_iri_only['AGE'] + fit_PATCH_AREA.params['Const']
        y_iri_only['POTHOLES_NUMBER'] = fit_POTHOLES_NUMBER.params['AGE'] * y_iri_only['AGE'] + fit_POTHOLES_NUMBER.params['Const']
        y_iri_only['POTHOLES_AREA'] = fit_POTHOLES_AREA.params['AGE'] * y_iri_only['AGE'] + fit_POTHOLES_AREA.params['Const']
        y_iri_only['SHOVING_NO'] = fit_SHOVING_NO.params['AGE'] * y_iri_only['AGE'] + fit_SHOVING_NO.params['Const']
        y_iri_only['SHOVING_A'] = fit_SHOVING_A.params['AGE'] * y_iri_only['AGE'] + fit_SHOVING_A.params['Const']
        y_iri_only['BLEEDING'] = fit_BLEEDING.params['AGE'] * y_iri_only['AGE'] + fit_BLEEDING.params['Const']
        y_iri_only['POLISH_AGG_A'] = fit_POLISH_AGG_A.params['AGE'] * y_iri_only['AGE'] + fit_POLISH_AGG_A.params['Const']
        y_iri_only['RAVELING'] = fit_RAVELING.params['AGE'] * y_iri_only['AGE'] + fit_RAVELING.params['Const']
        y_iri_only['PUMPING_NO'] = fit_PUMPING_NO.params['AGE'] * y_iri_only['AGE'] + fit_PUMPING_NO.params['Const']
        y_iri_only['PUMPING_L'] = fit_PUMPING_L.params['AGE'] * y_iri_only['AGE'] + fit_PUMPING_L.params['Const']
        y_iri_only['WP_LENGTH_CRACKED'] = fit_WP_LENGTH_CRACKED.params['AGE'] * y_iri_only['AGE'] + fit_WP_LENGTH_CRACKED.params['Const']
        y_iri_only['TRANS_CRACK_L_GT183'] = fit_TRANS_CRACK_L_GT183.params['AGE'] * y_iri_only['AGE'] + fit_TRANS_CRACK_L_GT183.params['Const']
    except:
        y_iri_only['GATOR_CRACK'] = df_5_sps1['GATOR_CRACK'].mean()
        y_iri_only['BLOCK_CRACK'] = df_5_sps1['BLOCK_CRACK'].mean()
        y_iri_only['EDGE_CRACK'] = df_5_sps1['EDGE_CRACK'].mean()
        y_iri_only['LONGITUDINAL_CRACK_WP'] = df_5_sps1['LONGITUDINAL_CRACK_WP'].mean()
        y_iri_only['LONGITUDINAL_CRACK_NWP'] = df_5_sps1['LONGITUDINAL_CRACK_NWP'].mean()
        y_iri_only['TRANS_CRACK_NUMBER'] = df_5_sps1['TRANS_CRACK_NUMBER'].mean()
        y_iri_only['TRANS_CRACK_LENGTH'] = df_5_sps1['TRANS_CRACK_LENGTH'].mean()
        y_iri_only['PATCH_NUMBER'] = df_5_sps1['PATCH_NUMBER'].mean()
        y_iri_only['PATCH_AREA'] = df_5_sps1['PATCH_AREA'].mean()
        y_iri_only['POTHOLES_NUMBER'] = df_5_sps1['POTHOLES_NUMBER'].mean()
        y_iri_only['POTHOLES_AREA'] = df_5_sps1['POTHOLES_AREA'].mean()
        y_iri_only['SHOVING_NO'] = df_5_sps1['SHOVING_NO'].mean()
        y_iri_only['SHOVING_A'] = df_5_sps1['SHOVING_A'].mean()
        y_iri_only['BLEEDING'] = df_5_sps1['BLEEDING'].mean()
        y_iri_only['POLISH_AGG_A'] = df_5_sps1['POLISH_AGG_A'].mean()
        y_iri_only['RAVELING'] = df_5_sps1['RAVELING'].mean()
        y_iri_only['PUMPING_NO'] = df_5_sps1['PUMPING_NO'].mean()
        y_iri_only['PUMPING_L'] = df_5_sps1['PUMPING_L'].mean()
        y_iri_only['WP_LENGTH_CRACKED'] = df_5_sps1['WP_LENGTH_CRACKED'].mean()
        y_iri_only['TRANS_CRACK_L_GT183'] = df_5_sps1['TRANS_CRACK_L_GT183'].mean()
        
        
    # rutting
    try:
        y_rutting_only = y[y['EVENT'] == 'RUTTING']

        Y_RUTTING = y_rutting_only['RUTTING']
        X_RUTTING = pd.DataFrame({'Const':1,
                          'AGE':y_rutting_only['AGE']})
        # Describe the linear model
        mod_RUTTING = sm.OLS(Y_RUTTING,X_RUTTING)

        # Fit model
        fit_RUTTING = mod_RUTTING.fit()

        y_iri_only['RUTTING'] = fit_RUTTING.params['AGE'] * y_iri_only['AGE'] + fit_RUTTING.params['Const']
    except:
        y_iri_only['RUTTING'] = df_5_sps1['RUTTING'].mean()
        
    # friction
    try:
        y_friction_only = y[y['EVENT'] == 'FRICTION']

        Y_FRICTION = y_friction_only['FRICTION']
        X_FRICTION = pd.DataFrame({'Const':1,
                          'AGE':y_friction_only['AGE']})
        # Describe the linear model
        mod_FRICTION = sm.OLS(Y_FRICTION,X_FRICTION)

        # Fit model
        fit_FRICTION = mod_FRICTION.fit()

        y_iri_only['FRICTION'] = fit_FRICTION.params['AGE'] * y_iri_only['AGE'] + fit_FRICTION.params['Const']
    except:
        y_iri_only['FRICTION'] = df_5_sps1['FRICTION'].mean()
        
    # drainage
    try:
        y_drainage_only = y[y['EVENT'] == 'DRAINAGE']

        Y_DRAINAGE = y_drainage_only['HYDRAULIC_CONDUCTIVITY']
        X_DRAINAGE = pd.DataFrame({'Const':1,
                          'AGE':y_drainage_only['AGE']})
        # Describe the linear model
        mod_DRAINAGE = sm.OLS(Y_DRAINAGE,X_DRAINAGE)

        # Fit model
        fit_DRAINAGE = mod_DRAINAGE.fit()

        y_iri_only['HYDRAULIC_CONDUCTIVITY'] = fit_DRAINAGE.params['AGE'] * y_iri_only['AGE'] + fit_DRAINAGE.params['Const']
    except:
        y_iri_only['HYDRAULIC_CONDUCTIVITY'] = df_5_sps1['HYDRAULIC_CONDUCTIVITY'].mean()
    
    # FWD
    try:
        y_fwd_only = y[y['EVENT'] == 'FWD']

        Y_BC_LAYER_NO_1_MODULUS = y_fwd_only['BC_LAYER_NO_1_MODULUS']
        Y_BC_LAYER_NO_2_MODULUS = y_fwd_only['BC_LAYER_NO_2_MODULUS']
        Y_BC_LAYER_NO_3_MODULUS = y_fwd_only['BC_LAYER_NO_3_MODULUS']
        Y_BC_LAYER_NO_4_MODULUS = y_fwd_only['BC_LAYER_NO_4_MODULUS']
        Y_BC_LAYER_NO_5_MODULUS = y_fwd_only['BC_LAYER_NO_5_MODULUS']


        X_FWD = pd.DataFrame({'Const':1,
                          'AGE':y_fwd_only['AGE']})
        # Describe the linear model
        mod_BC_LAYER_NO_1_MODULUS = sm.OLS(Y_BC_LAYER_NO_1_MODULUS,X_FWD)
        mod_BC_LAYER_NO_2_MODULUS = sm.OLS(Y_BC_LAYER_NO_2_MODULUS,X_FWD)
        mod_BC_LAYER_NO_3_MODULUS = sm.OLS(Y_BC_LAYER_NO_3_MODULUS,X_FWD)
        mod_BC_LAYER_NO_4_MODULUS = sm.OLS(Y_BC_LAYER_NO_4_MODULUS,X_FWD)
        mod_BC_LAYER_NO_5_MODULUS = sm.OLS(Y_BC_LAYER_NO_5_MODULUS,X_FWD)

        # Fit model
        fit_BC_LAYER_NO_1_MODULUS = mod_BC_LAYER_NO_1_MODULUS.fit()
        fit_BC_LAYER_NO_2_MODULUS = mod_BC_LAYER_NO_2_MODULUS.fit()
        fit_BC_LAYER_NO_3_MODULUS = mod_BC_LAYER_NO_3_MODULUS.fit()
        fit_BC_LAYER_NO_4_MODULUS = mod_BC_LAYER_NO_4_MODULUS.fit()
        fit_BC_LAYER_NO_5_MODULUS = mod_BC_LAYER_NO_5_MODULUS.fit()

        y_iri_only['BC_LAYER_NO_1_MODULUS'] = fit_BC_LAYER_NO_1_MODULUS.params['AGE'] * y_iri_only['AGE'] + fit_BC_LAYER_NO_1_MODULUS.params['Const']
        y_iri_only['BC_LAYER_NO_2_MODULUS'] = fit_BC_LAYER_NO_2_MODULUS.params['AGE'] * y_iri_only['AGE'] + fit_BC_LAYER_NO_2_MODULUS.params['Const']
        y_iri_only['BC_LAYER_NO_3_MODULUS'] = fit_BC_LAYER_NO_3_MODULUS.params['AGE'] * y_iri_only['AGE'] + fit_BC_LAYER_NO_3_MODULUS.params['Const']
        y_iri_only['BC_LAYER_NO_4_MODULUS'] = fit_BC_LAYER_NO_4_MODULUS.params['AGE'] * y_iri_only['AGE'] + fit_BC_LAYER_NO_4_MODULUS.params['Const']
        y_iri_only['BC_LAYER_NO_5_MODULUS'] = fit_BC_LAYER_NO_5_MODULUS.params['AGE'] * y_iri_only['AGE'] + fit_BC_LAYER_NO_5_MODULUS.params['Const']
    except:
        y_iri_only['BC_LAYER_NO_1_MODULUS'] = df_5_sps1['BC_LAYER_NO_1_MODULUS'].mean()
        y_iri_only['BC_LAYER_NO_2_MODULUS'] = df_5_sps1['BC_LAYER_NO_2_MODULUS'].mean()
        y_iri_only['BC_LAYER_NO_3_MODULUS'] = df_5_sps1['BC_LAYER_NO_3_MODULUS'].mean()
        y_iri_only['BC_LAYER_NO_4_MODULUS'] = df_5_sps1['BC_LAYER_NO_4_MODULUS'].mean()
        y_iri_only['BC_LAYER_NO_5_MODULUS'] = df_5_sps1['BC_LAYER_NO_5_MODULUS'].mean()
        
    df_6 = df_6.append(y_iri_only)

In [50]:
df_6

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,CN_ASSIGN_DATE,ASSIGN_DATE,IMP_TYPE,IRI_VISIT_NO,MEAN_IRI_LEFT_WHEEL_PATH,MEAN_IRI_RIGHT_WHEEL_PATH,MEAN_MRI,IRI_VISIT_DATE,EVENT,DISTRESS_SURVEY_DATE,GATOR_CRACK,BLOCK_CRACK,EDGE_CRACK,LONGITUDINAL_CRACK_WP,LONGITUDINAL_CRACK_NWP,TRANS_CRACK_NUMBER,TRANS_CRACK_LENGTH,PATCH_NUMBER,PATCH_AREA,POTHOLES_NUMBER,POTHOLES_AREA,SHOVING_NO,SHOVING_A,BLEEDING,POLISH_AGG_A,RAVELING,PUMPING_NO,PUMPING_L,WP_LENGTH_CRACKED,TRANS_CRACK_L_GT183,RUTTING_VISIT_NO,RUTTING_SURVEY_DATE,RUTTING,FRICTION_DATE,FRICTION,DRAINAGE_TEST_DATE,PVMT_THICK_DB,PATB_THICK_DB,HYDRAULIC_CONDUCTIVITY,MODULUS_TEST_DATE,BC_LAYER_NO_1_MODULUS,BC_LAYER_NO_2_MODULUS,BC_LAYER_NO_3_MODULUS,BC_LAYER_NO_4_MODULUS,BC_LAYER_NO_5_MODULUS,AGE,INITIAL_IRI
13,1,0101,1.0,1995-10-30,NaT,1991-04-30,NaN,01010101,0.6572,0.6574,0.6572,1995-10-30,IRI,NaT,-3.932266,0.0,0.0,12.365742,0.173128,-3.523890,-1.443286,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,20.049527,-0.047778,0.190046,-0.248165,-0.149438,NaN,NaT,6.435719,NaT,37.814286,NaT,NaN,NaN,9945.685558,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,4.501119,0.635183
16,1,0101,1.0,1996-01-10,NaT,1991-04-30,NaN,01010102,0.6962,0.6384,0.6670,1996-01-10,IRI,NaT,-2.481933,0.0,0.0,12.247009,0.170197,-2.813874,-1.158582,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.864897,-0.028871,0.240824,4.238165,-0.122338,NaN,NaT,6.466592,NaT,36.785714,NaT,NaN,NaN,9945.685558,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,4.698248,0.635183
21,1,0101,1.0,1996-04-08,NaT,1991-04-30,NaN,01010103,0.6508,0.7148,0.6828,1996-04-08,IRI,NaT,-0.689160,0.0,0.0,12.100242,0.166574,-1.936216,-0.806657,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.636674,-0.005501,0.303591,9.783768,-0.088840,NaN,NaT,6.504755,NaT,35.514286,NaT,NaN,NaN,9945.685558,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,4.941922,0.635183
27,1,0101,1.0,1996-08-12,NaT,1991-04-30,NaN,01010104,0.6804,0.7132,0.6970,1996-08-12,IRI,NaT,1.848923,0.0,0.0,11.892460,0.161444,-0.693688,-0.308425,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.313571,0.027586,0.392453,17.634846,-0.041416,NaN,NaT,6.558783,NaT,33.714286,NaT,NaN,NaN,9945.685558,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,5.286898,0.635183
33,1,0101,1.0,1996-10-11,NaT,1991-04-30,NaN,01010105,0.6836,0.6724,0.6780,1996-10-11,IRI,NaT,3.057534,0.0,0.0,11.793516,0.159002,-0.102008,-0.071172,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.159712,0.043341,0.434768,21.373454,-0.018832,NaN,NaT,6.584510,NaT,32.857143,NaT,NaN,NaN,9945.685558,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,5.451173,0.635183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11047,55,0124,1.0,2002-05-10,NaT,1997-01-01,NaN,55012406,0.9748,1.1158,1.0454,2002-05-10,IRI,NaT,0.000000,0.0,0.0,0.000000,168.834606,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,102.013172,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaT,8.205664,NaT,46.921053,NaT,NaN,NaN,4868.161670,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,5.352608,0.848896
11049,55,0124,1.0,2003-04-15,NaT,1997-01-01,NaN,55012407,1.1232,1.1440,1.1334,2003-04-15,IRI,NaT,0.000000,0.0,0.0,0.000000,211.042519,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,138.432552,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaT,8.977366,NaT,46.921053,NaT,NaN,NaN,5690.908587,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,6.283497,0.848896
11055,55,0124,1.0,2004-10-19,NaT,1997-01-01,NaN,55012408,1.1086,1.1562,1.1324,2004-10-19,IRI,NaT,0.000000,0.0,0.0,0.000000,279.692448,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,197.667602,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaT,10.232517,NaT,46.921053,NaT,NaN,NaN,7029.082248,NaN,1195.443398,519.017952,127.704923,52.343468,71.516652,7.797559,0.848896
11058,55,0124,1.0,2005-09-12,NaT,1997-01-01,NaN,55012409,1.1380,1.1464,1.1424,2005-09-12,IRI,NaT,0.00000

In [51]:
df_6 = df_6[['STATE_CODE', 'SHRP_ID', 'CONSTRUCTION_NO', 'DATE','ASSIGN_DATE', 'MEAN_IRI_LEFT_WHEEL_PATH','MEAN_IRI_RIGHT_WHEEL_PATH', 'MEAN_MRI',\
             'GATOR_CRACK', 'BLOCK_CRACK', 'EDGE_CRACK','LONGITUDINAL_CRACK_WP', 'LONGITUDINAL_CRACK_NWP', 'TRANS_CRACK_NUMBER',\
             'TRANS_CRACK_LENGTH', 'PATCH_NUMBER', 'PATCH_AREA', 'POTHOLES_NUMBER','POTHOLES_AREA', 'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A',\
             'RAVELING', 'PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED', 'TRANS_CRACK_L_GT183', 'RUTTING', 'FRICTION',\
             'HYDRAULIC_CONDUCTIVITY','BC_LAYER_NO_1_MODULUS', 'BC_LAYER_NO_2_MODULUS','BC_LAYER_NO_3_MODULUS', 'BC_LAYER_NO_4_MODULUS','BC_LAYER_NO_5_MODULUS', 'AGE', 'INITIAL_IRI']]

In [52]:
df_6.columns

Index(['STATE_CODE', 'SHRP_ID', 'CONSTRUCTION_NO', 'DATE', 'ASSIGN_DATE',
       'MEAN_IRI_LEFT_WHEEL_PATH', 'MEAN_IRI_RIGHT_WHEEL_PATH', 'MEAN_MRI',
       'GATOR_CRACK', 'BLOCK_CRACK', 'EDGE_CRACK', 'LONGITUDINAL_CRACK_WP',
       'LONGITUDINAL_CRACK_NWP', 'TRANS_CRACK_NUMBER', 'TRANS_CRACK_LENGTH',
       'PATCH_NUMBER', 'PATCH_AREA', 'POTHOLES_NUMBER', 'POTHOLES_AREA',
       'SHOVING_NO', 'SHOVING_A', 'BLEEDING', 'POLISH_AGG_A', 'RAVELING',
       'PUMPING_NO', 'PUMPING_L', 'WP_LENGTH_CRACKED', 'TRANS_CRACK_L_GT183',
       'RUTTING', 'FRICTION', 'HYDRAULIC_CONDUCTIVITY',
       'BC_LAYER_NO_1_MODULUS', 'BC_LAYER_NO_2_MODULUS',
       'BC_LAYER_NO_3_MODULUS', 'BC_LAYER_NO_4_MODULUS',
       'BC_LAYER_NO_5_MODULUS', 'AGE', 'INITIAL_IRI'],
      dtype='object')

## structure

In [61]:
data_sps1_layer = pd.read_sql("select * from SPS1_LAYER", conn_sps)
data_sps1_layer_thickness = pd.read_sql("select * from SPS1_LAYER_THICKNESS", conn_sps)

In [62]:
data_sps1_layer_thickness.columns

Index(['SHRP_ID', 'STATE_CODE', 'STATION_NO', 'OFFSET', 'DENSE_GRADE_AGG_BASE',
       'PERM_ASPH_TREAT_BASE', 'DENSE_GRD_ASPH_TREAT_BASE',
       'SURFACE_AND_BINDER', 'SURFACE_FRICTION', 'RECORD_STATUS'],
      dtype='object')

In [66]:
data_sps1_layer_thickness_mean = data_sps1_layer_thickness.groupby(['STATE_CODE','SHRP_ID'])[['DENSE_GRADE_AGG_BASE','PERM_ASPH_TREAT_BASE',\
                                                                                      'DENSE_GRD_ASPH_TREAT_BASE','SURFACE_AND_BINDER','SURFACE_FRICTION']].mean()
data_sps1_layer_thickness_mean = data_sps1_layer_thickness_mean.reset_index()
data_sps1_layer_thickness_mean.loc[data_sps1_layer_thickness_mean['DENSE_GRADE_AGG_BASE'].isna(),'DENSE_GRADE_AGG_BASE'] = 0
data_sps1_layer_thickness_mean.loc[data_sps1_layer_thickness_mean['PERM_ASPH_TREAT_BASE'].isna(),'PERM_ASPH_TREAT_BASE'] = 0
data_sps1_layer_thickness_mean.loc[data_sps1_layer_thickness_mean['DENSE_GRD_ASPH_TREAT_BASE'].isna(),'DENSE_GRD_ASPH_TREAT_BASE'] = 0
data_sps1_layer_thickness_mean.loc[data_sps1_layer_thickness_mean['SURFACE_AND_BINDER'].isna(),'SURFACE_AND_BINDER'] = 0
data_sps1_layer_thickness_mean.loc[data_sps1_layer_thickness_mean['SURFACE_FRICTION'].isna(),'SURFACE_FRICTION'] = 0


In [77]:
data_sps1_layer_thickness_mean.isnull().sum()

STATE_CODE                   0
SHRP_ID                      0
DENSE_GRADE_AGG_BASE         0
PERM_ASPH_TREAT_BASE         0
DENSE_GRD_ASPH_TREAT_BASE    0
SURFACE_AND_BINDER           0
SURFACE_FRICTION             0
dtype: int64

In [214]:
df_7 = pd.merge(df_6,data_sps1_layer_thickness_mean[['STATE_CODE','SHRP_ID','DENSE_GRADE_AGG_BASE','PERM_ASPH_TREAT_BASE',\
                                         'DENSE_GRD_ASPH_TREAT_BASE','SURFACE_AND_BINDER','SURFACE_FRICTION']],how='left',on=['STATE_CODE','SHRP_ID'])

## climate

In [215]:
data_aws_link = pd.read_sql("select * from AWS_LINK", conn_aws)
data_aws_humidity_month = pd.read_sql("select * from AWS_HUMIDITY_MONTH", conn_aws)
data_aws_precipitation_month = pd.read_sql("select * from AWS_PRECIPITATION_MONTH", conn_aws)
data_aws_solar_month = pd.read_sql("select * from AWS_SOLAR_MONTH", conn_aws)
data_aws_temp_month = pd.read_sql("select * from AWS_TEMP_MONTH", conn_aws)
data_aws_wind_month = pd.read_sql("select * from AWS_WIND_MONTH", conn_aws)

### humidity

In [216]:
import calendar
data_aws_humidity_month['START_TIME'] = pd.to_datetime(data_aws_humidity_month['YEAR'].apply(str)+'-'+data_aws_humidity_month['MONTH'].apply(str)+'-1', format='%Y-%m-%d')

last_day = data_aws_humidity_month.apply(lambda row: calendar.monthrange(row['YEAR'],row['MONTH'])[1], axis = 1).apply(str)
data_aws_humidity_month['END_TIME'] = pd.to_datetime(data_aws_humidity_month['YEAR'].apply(str)+'-'+data_aws_humidity_month['MONTH'].apply(str)+'-'+last_day, format='%Y-%m-%d')

In [217]:
data_aws_humidity_month_merged = pd.merge(data_aws_humidity_month,data_aws_link[['STATE_CODE','SHRP_ID','AWS_ID']],how='left',on='AWS_ID')
data_aws_humidity_month_merged

,AWS_ID,YEAR,MONTH,MAX_MON_HUM_AVG,MAX_MON_HUM_STDEV,MIN_MON_HUM_AVG,MIN_MON_HUM_STDEV,NUMBER_MON_HUMID_DAYS,RECORD_STATUS,START_TIME,END_TIME,STATE_CODE,SHRP_ID
0,53A800,2002,6,81.9,11.8,22.1,10.5,30,E,2002-06-01,2002-06-30,53,A800
1,53A800,2002,7,67.3,11.0,15.0,4.5,31,E,2002-07-01,2002-07-31,53,A800
2,040200,2002,5,22.9,6.5,6.8,1.8,31,E,2002-05-01,2002-05-31,4,0200
3,040200,2002,5,22.9,6.5,6.8,1.8,31,E,2002-05-01,2002-05-31,4,B900
4,040200,2002,5,22.9,6.5,6.8,1.8,31,E,2002-05-01,2002-05-31,4,1003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7936,370800,2002,4,96.6,2.7,41.4,13.2,24,E,2002-04-01,2002-04-30,37,0800
7937,370800,2002,5,97.4,3.5,44.2,14.3,31,E,2002-05-01,2002-05-31,37,0800
7938,370800,2002,6,98.4,2.7,48.1,15.0,30,E,2002-06-01,2002-06-30,37,0800
7939,370800,2002,7,98.7,1.7,52.3,11.5,31,E,2002-07-01,2002-07-31,37,0800


In [218]:
df_7['MAX_MON_HUM_AVG'] = None
df_7['MIN_MON_HUM_AVG'] = None

for i in df_7.index:
# i=2968
    data_aws_humidity_month_merged_sub = data_aws_humidity_month_merged[(data_aws_humidity_month_merged['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_aws_humidity_month_merged['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_aws_humidity_month_merged_sub = data_aws_humidity_month_merged_sub[(data_aws_humidity_month_merged_sub['START_TIME'] > df_7.loc[i,'ASSIGN_DATE']) & \
                                       (data_aws_humidity_month_merged_sub['END_TIME'] < df_7.loc[i,'DATE'])] 

    df_7.loc[i,'MAX_MON_HUM_AVG'] = data_aws_humidity_month_merged_sub['MAX_MON_HUM_AVG'].mean() if len(data_aws_humidity_month_merged_sub) > 0 else data_aws_humidity_month_merged['MAX_MON_HUM_AVG'].mean()
    df_7.loc[i,'MIN_MON_HUM_AVG'] = data_aws_humidity_month_merged_sub['MIN_MON_HUM_AVG'].mean() if len(data_aws_humidity_month_merged_sub) > 0 else data_aws_humidity_month_merged['MIN_MON_HUM_AVG'].mean()

### precipitation

In [219]:
data_aws_precipitation_month = pd.read_sql("select * from AWS_PRECIPITATION_MONTH", conn_aws)

In [220]:
data_aws_precipitation_month['START_TIME'] = pd.to_datetime(data_aws_precipitation_month['YEAR'].apply(str)+'-'+data_aws_precipitation_month['MONTH'].apply(str)+'-1', format='%Y-%m-%d')

last_day = data_aws_precipitation_month.apply(lambda row: calendar.monthrange(row['YEAR'],row['MONTH'])[1], axis = 1).apply(str)
data_aws_precipitation_month['END_TIME'] = pd.to_datetime(data_aws_precipitation_month['YEAR'].apply(str)+'-'+data_aws_precipitation_month['MONTH'].apply(str)+'-'+last_day, format='%Y-%m-%d')

In [221]:
data_aws_precipitation_month_merged = pd.merge(data_aws_precipitation_month,data_aws_link[['STATE_CODE','SHRP_ID','AWS_ID']],how='left',on='AWS_ID')
data_aws_precipitation_month_merged

,AWS_ID,YEAR,MONTH,TOTAL_MON_PRECIP,INTENSE_PRECIP_DAYS,WET_DAYS,NUMBER_MON_PRECIP_HOURS,RECORD_STATUS,START_TIME,END_TIME,STATE_CODE,SHRP_ID
0,530200,2007,9,8,0,3,720,E,2007-09-01,2007-09-30,53,0200
1,530200,2007,9,8,0,3,720,E,2007-09-01,2007-09-30,53,1005
2,530800,2007,8,14,0,3,744,E,2007-08-01,2007-08-31,53,0800
3,530800,2007,9,7,0,2,720,E,2007-09-01,2007-09-30,53,0800
4,530800,2007,10,31,0,9,744,E,2007-10-01,2007-10-31,53,0800
...,...,...,...,...,...,...,...,...,...,...,...,...
9167,510100,2002,12,120,4,11,744,E,2002-12-01,2002-12-31,51,0100
9168,100100,2003,5,117,2,18,688,E,2003-05-01,2003-05-31,10,0200
9169,100100,2003,5,117,2,18,688,E,2003-05-01,2003-05-31,10,0100
9170,100100,2003,6,105,2,19,720,E,2003-06-01,2003-06-30,10,0200


In [222]:
df_7['TOTAL_MON_PRECIP'] = None

for i in df_7.index:
# i=2968
    data_aws_precipitation_month_merged_sub = data_aws_precipitation_month_merged[(data_aws_precipitation_month_merged['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_aws_precipitation_month_merged['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_aws_precipitation_month_merged_sub = data_aws_precipitation_month_merged_sub[(data_aws_precipitation_month_merged_sub['START_TIME'] > df_7.loc[i,'ASSIGN_DATE']) & \
                                       (data_aws_precipitation_month_merged_sub['END_TIME'] < df_7.loc[i,'DATE'])] 

    df_7.loc[i,'TOTAL_MON_PRECIP'] = data_aws_precipitation_month_merged_sub['TOTAL_MON_PRECIP'].mean() \
    if len(data_aws_precipitation_month_merged_sub) > 0 else data_aws_precipitation_month_merged['TOTAL_MON_PRECIP'].mean()

### solar

In [223]:
data_aws_solar_month = pd.read_sql("select * from AWS_SOLAR_MONTH", conn_aws)

In [224]:
data_aws_solar_month['START_TIME'] = pd.to_datetime(data_aws_solar_month['YEAR'].apply(str)+'-'+data_aws_solar_month['MONTH'].apply(str)+'-1', format='%Y-%m-%d')

last_day = data_aws_solar_month.apply(lambda row: calendar.monthrange(row['YEAR'],row['MONTH'])[1], axis = 1).apply(str)
data_aws_solar_month['END_TIME'] = pd.to_datetime(data_aws_solar_month['YEAR'].apply(str)+'-'+data_aws_solar_month['MONTH'].apply(str)+'-'+last_day, format='%Y-%m-%d')

In [225]:
data_aws_solar_month_merged = pd.merge(data_aws_solar_month,data_aws_link[['STATE_CODE','SHRP_ID','AWS_ID']],how='left',on='AWS_ID')
data_aws_solar_month_merged

,AWS_ID,YEAR,MONTH,MEAN_MON_SOLAR_AVG,MEAN_MON_SOLAR_STDEV,NUMBER_MON_SOLAR_DAYS,RECORD_STATUS,START_TIME,END_TIME,STATE_CODE,SHRP_ID
0,490800,2005,9,224.8,48.8,30,E,2005-09-01,2005-09-30,49,0800
1,490800,2005,10,150.0,52.7,31,E,2005-10-01,2005-10-31,49,0800
2,530200,2005,9,172.1,48.6,30,E,2005-09-01,2005-09-30,53,0200
3,530200,2005,9,172.1,48.6,30,E,2005-09-01,2005-09-30,53,1005
4,530200,2005,10,102.1,32.9,31,E,2005-10-01,2005-10-31,53,0200
...,...,...,...,...,...,...,...,...,...,...,...
8654,360800,2003,5,212.0,89.7,31,E,2003-05-01,2003-05-31,36,0800
8655,360800,2003,6,234.0,88.9,30,E,2003-06-01,2003-06-30,36,0800
8656,360800,2003,7,238.8,78.8,31,E,2003-07-01,2003-07-31,36,0800
8657,360800,2003,8,210.0,52.7,31,E,2003-08-01,2003-08-31,36,0800


In [226]:
df_7['MEAN_MON_SOLAR_AVG'] = None

for i in df_7.index:
# i=2968
    data_aws_solar_month_merged_sub = data_aws_solar_month_merged[(data_aws_solar_month_merged['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_aws_solar_month_merged['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_aws_solar_month_merged_sub = data_aws_solar_month_merged_sub[(data_aws_solar_month_merged_sub['START_TIME'] > df_7.loc[i,'ASSIGN_DATE']) & \
                                       (data_aws_solar_month_merged_sub['END_TIME'] < df_7.loc[i,'DATE'])] 

    df_7.loc[i,'MEAN_MON_SOLAR_AVG'] = data_aws_solar_month_merged_sub['MEAN_MON_SOLAR_AVG'].mean() \
    if len(data_aws_solar_month_merged_sub) > 0 else data_aws_solar_month_merged['MEAN_MON_SOLAR_AVG'].mean()

### temp

In [227]:
data_aws_temp_month = pd.read_sql("select * from AWS_TEMP_MONTH", conn_aws)

In [228]:
data_aws_temp_month['START_TIME'] = pd.to_datetime(data_aws_temp_month['YEAR'].apply(str)+'-'+data_aws_temp_month['MONTH'].apply(str)+'-1', format='%Y-%m-%d')

last_day = data_aws_temp_month.apply(lambda row: calendar.monthrange(row['YEAR'],row['MONTH'])[1], axis = 1).apply(str)
data_aws_temp_month['END_TIME'] = pd.to_datetime(data_aws_temp_month['YEAR'].apply(str)+'-'+data_aws_temp_month['MONTH'].apply(str)+'-'+last_day, format='%Y-%m-%d')

In [229]:
data_aws_temp_month_merged = pd.merge(data_aws_temp_month,data_aws_link[['STATE_CODE','SHRP_ID','AWS_ID']],how='left',on='AWS_ID')
data_aws_temp_month_merged

,AWS_ID,YEAR,MONTH,MEAN_MON_TEMP_AVG,MEAN_MON_TEMP_STDEV,MAX_MON_TEMP_AVG,MAX_MON_TEMP_STDEV,MIN_MON_TEMP_AVG,MIN_MON_TEMP_STDEV,MAX_MON_TEMP,MIN_MON_TEMP,DAYS_ABOVE_32_C,DAYS_BELOW_0_C,FREEZE_INDEX,FREEZE_THAW,NUMBER_MON_TEMP_DAYS,RECORD_STATUS,START_TIME,END_TIME,STATE_CODE,SHRP_ID
0,390200,1998,10,11.8,3.7,18.7,4.4,5.3,4.4,29.0,-2.1,0,3,0,3,31,E,1998-10-01,1998-10-31,39,0200
1,390200,1998,10,11.8,3.7,18.7,4.4,5.3,4.4,29.0,-2.1,0,3,0,3,31,E,1998-10-01,1998-10-31,39,0100
2,390200,1998,10,11.8,3.7,18.7,4.4,5.3,4.4,29.0,-2.1,0,3,0,3,31,E,1998-10-01,1998-10-31,39,0800
3,390200,1998,10,11.8,3.7,18.7,4.4,5.3,4.4,29.0,-2.1,0,3,0,3,31,E,1998-10-01,1998-10-31,39,0900
4,390200,1998,11,6.6,3.6,11.9,4.2,0.8,4.1,19.8,-4.0,0,19,0,19,30,E,1998-11-01,1998-11-30,39,0200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9414,380200,1999,5,14.3,4.4,19.9,5.7,8.5,3.8,32.0,0.8,0,0,0,0,31,E,1999-05-01,1999-05-31,38,5002
9415,390200,1998,9,19.7,3.6,27.9,4.2,12.2,4.7,35.2,3.0,4,0,0,0,30,E,1998-09-01,1998-09-30,39,0200
9416,390200,1998,9,19.7,3.6,27.9,4.2,12.2,4.7,35.2,3.0,4,0,0,0,30,E,1998-09-01,1998-09-30,39,0100
9417,390200,1998,9,19.7,3.6,27.9,4.2,12.2,4.7,35.2,3.0,4,0,0,0,30,E,1998-09-01,1998-09-30,39,0800


In [230]:
df_7['MAX_MON_TEMP_AVG'] = None
df_7['MIN_MON_TEMP_AVG'] = None
df_7['DAYS_ABOVE_32_C'] = None
df_7['DAYS_BELOW_0_C'] = None
df_7['FREEZE_INDEX'] = None

for i in df_7.index:
# i=2968
    data_aws_temp_month_merged_sub = data_aws_temp_month_merged[(data_aws_temp_month_merged['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_aws_temp_month_merged['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_aws_temp_month_merged_sub = data_aws_temp_month_merged_sub[(data_aws_temp_month_merged_sub['START_TIME'] > df_7.loc[i,'ASSIGN_DATE']) & \
                                       (data_aws_temp_month_merged_sub['END_TIME'] < df_7.loc[i,'DATE'])] 

    df_7.loc[i,'MAX_MON_TEMP_AVG'] = data_aws_temp_month_merged_sub['MAX_MON_TEMP_AVG'].mean() \
    if len(data_aws_temp_month_merged_sub) > 0 else data_aws_temp_month_merged['MAX_MON_TEMP_AVG'].mean()
    
    df_7.loc[i,'MIN_MON_TEMP_AVG'] = data_aws_temp_month_merged_sub['MIN_MON_TEMP_AVG'].mean() \
    if len(data_aws_temp_month_merged_sub) > 0 else data_aws_temp_month_merged['MIN_MON_TEMP_AVG'].mean()
    
    df_7.loc[i,'DAYS_ABOVE_32_C'] = data_aws_temp_month_merged_sub['DAYS_ABOVE_32_C'].mean() \
    if len(data_aws_temp_month_merged_sub) > 0 else data_aws_temp_month_merged['DAYS_ABOVE_32_C'].mean()
    
    df_7.loc[i,'DAYS_BELOW_0_C'] = data_aws_temp_month_merged_sub['DAYS_BELOW_0_C'].mean() \
    if len(data_aws_temp_month_merged_sub) > 0 else data_aws_temp_month_merged['DAYS_BELOW_0_C'].mean()
    
    df_7.loc[i,'FREEZE_INDEX'] = data_aws_temp_month_merged_sub['FREEZE_INDEX'].mean() \
    if len(data_aws_temp_month_merged_sub) > 0 else data_aws_temp_month_merged['FREEZE_INDEX'].mean()

### wind

In [231]:
data_aws_wind_month = pd.read_sql("select * from AWS_WIND_MONTH", conn_aws)

In [232]:
data_aws_wind_month['START_TIME'] = pd.to_datetime(data_aws_wind_month['YEAR'].apply(str)+'-'+data_aws_wind_month['MONTH'].apply(str)+'-1', format='%Y-%m-%d')

last_day = data_aws_wind_month.apply(lambda row: calendar.monthrange(row['YEAR'],row['MONTH'])[1], axis = 1).apply(str)
data_aws_wind_month['END_TIME'] = pd.to_datetime(data_aws_wind_month['YEAR'].apply(str)+'-'+data_aws_wind_month['MONTH'].apply(str)+'-'+last_day, format='%Y-%m-%d')

In [233]:
data_aws_wind_month_merged = pd.merge(data_aws_wind_month,data_aws_link[['STATE_CODE','SHRP_ID','AWS_ID']],how='left',on='AWS_ID')
data_aws_wind_month_merged

,AWS_ID,YEAR,MONTH,MEAN_MON_WIND_AVG,MEAN_MON_WIND_STDEV,MAX_MON_WIND_SPD,NUMBER_MON_WIND_DAYS,RECORD_STATUS,START_TIME,END_TIME,STATE_CODE,SHRP_ID
0,040200,1996,8,2.4,0.7,23.3,31,E,1996-08-01,1996-08-31,4,0200
1,040200,1996,8,2.4,0.7,23.3,31,E,1996-08-01,1996-08-31,4,B900
2,040200,1996,8,2.4,0.7,23.3,31,E,1996-08-01,1996-08-31,4,1003
3,040200,1996,8,2.4,0.7,23.3,31,E,1996-08-01,1996-08-31,4,1006
4,040200,1996,8,2.4,0.7,23.3,31,E,1996-08-01,1996-08-31,4,1007
...,...,...,...,...,...,...,...,...,...,...,...,...
9354,510100,2002,10,0.5,0.2,7.8,31,E,2002-10-01,2002-10-31,51,0100
9355,100100,2002,11,0.7,0.4,8.9,30,E,2002-11-01,2002-11-30,10,0200
9356,100100,2002,11,0.7,0.4,8.9,30,E,2002-11-01,2002-11-30,10,0100
9357,100100,2002,12,0.9,0.5,10.4,30,E,2002-12-01,2002-12-31,10,0200


In [234]:
df_7['MEAN_MON_WIND_AVG'] = None

for i in df_7.index:
# i=2968
    data_aws_wind_month_merged_sub = data_aws_wind_month_merged[(data_aws_wind_month_merged['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_aws_wind_month_merged['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_aws_wind_month_merged_sub = data_aws_wind_month_merged_sub[(data_aws_wind_month_merged_sub['START_TIME'] > df_7.loc[i,'ASSIGN_DATE']) & \
                                       (data_aws_wind_month_merged_sub['END_TIME'] < df_7.loc[i,'DATE'])] 

    df_7.loc[i,'MEAN_MON_WIND_AVG'] = data_aws_wind_month_merged_sub['MEAN_MON_WIND_AVG'].mean() \
    if len(data_aws_wind_month_merged_sub) > 0 else data_aws_wind_month_merged['MEAN_MON_WIND_AVG'].mean()
    

## traffic

In [236]:
data_trf_esal_computed = pd.read_sql("select * from TRF_ESAL_COMPUTED", conn_trf)
data_trf_mon_est_esal = pd.read_sql("select * from TRF_MON_EST_ESAL", conn_trf)

In [237]:
data_trf_esal_computed

,STATE_CODE,SHRP_ID,YEAR,RECORD_STATUS,KESAL_YEAR
0,29,0702,1997,E,270
1,29,0702,1998,E,285
2,29,0703,1992,E,275
3,29,0703,1993,E,309
4,29,0703,1994,E,384
...,...,...,...,...,...
16012,29,0702,1992,E,261
16013,29,0702,1993,E,293
16014,29,0702,1994,E,361
16015,29,0702,1995,E,262


In [238]:
df_7['KESAL_YEAR'] = None
missing_traffic_section_counts= 0
for i in df_7.index:
# i=300
    data_trf_esal_computed_sub = data_trf_esal_computed[(data_trf_esal_computed['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                        (data_trf_esal_computed['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    data_trf_esal_computed_sub = data_trf_esal_computed_sub[(data_trf_esal_computed_sub['YEAR'] >= df_7.loc[i,'ASSIGN_DATE'].year) & \
                                       (data_trf_esal_computed_sub['YEAR'] <= df_7.loc[i,'DATE'].year)] 

    df_7.loc[i,'KESAL_YEAR'] = data_trf_esal_computed_sub['KESAL_YEAR'].mean() \
    if len(data_trf_esal_computed_sub) > 0 else data_trf_esal_computed['KESAL_YEAR'].mean()
    if len(data_trf_esal_computed_sub) == 0 : missing_traffic_section_counts+=1 # 785 out of 2968

## material

### pma_density_profile

In [243]:

data_sps1_pma_density_profile = pd.read_sql("select * from SPS1_PMA_DENSITY_PROFILE", conn_sps)
data_sps1_pma_density_profile

,SHRP_ID,STATE_CODE,ATB_MEASURE,ATB_NO_MEASUREMENTS,AVERAGE_ATB,MAXIMUM_ATB,MINIMUM_ATB,STD_DEV_ATB,LAYER_NO_ATB,BINDER_MEASURE,BINDER_NO_MEASUREMENTS,AVERAGE_BINDER,MAXIMUM_BINDER,MINIMUM_BINDER,STD_DEV_BINDER,LAYER_NO_BINDER,SURFACE_MEASURE,SURFACE_NO_MEASUREMENTS,AVERAGE_SURFACE,MAXIMUM_SURFACE,MINIMUM_SURFACE,STD_DEV_SURFACE,LAYER_NO_SURFACE,FRICTION_MEASURE,FRICTION_NO_MEASUREMENTS,AVERAGE_FRICTION,MAXIMUM_FRICTION,MINIMUM_FRICTION,STD_DEV_FRICTION,LAYER_NO_FRICTION,NUCLEAR_GAUGE_MAKER,GAUGE_MODEL_NUMBER,GAUGE_ID_NUMBER,GAUGE_COUNT_RATE,PROFILOGRAPH_TYPE,PROFILE_INDEX,INTERP_METHOD,BLANK_BAND_HEIGHT,CUTOFF_HEIGHT,INCENTIVE_PAYMENT,RECORD_STATUS
0,0113,51,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,4.0,A,12.0,130.8,136.9,127.3,3.0,5,None,None,None,None,None,None,None,TROXLER,4640-B,509533,5389.0,NaN,NaN,NaN,NaN,NaN,N,E
1,0114,51,None,NaN,NaN,NaN,NaN,NaN,NaN,A,12.0,135.3,136.7,133.6,1.2,4.0,A,12.0,127.2,129.9,120.1,2.7,5,None,None,None,None,None,None,None,TROXLER,4640-B,509533,5389.0,NaN,NaN,NaN,NaN,NaN,N,E
2,0115,51,A,12.0,126.4,131.1,122.1,2.9,3.0,None,NaN,NaN,NaN,NaN,NaN,4.0,A,12.0,134.8,137.2,132.1,1.5,5,None,None,None,None,None,None,None,TROXLER,4640-B,509533,5389.0,NaN,NaN,NaN,NaN,NaN,N,E
3,0116,51,A,12.0,134.3,137.7,129.1,2.3,3.0,None,NaN,NaN,NaN,NaN,NaN,4.0,A,12.0,132.4,136.2,126.6,2.6,5,None,None,None,None,None,None,None,TROXLER,4640-B,509533,5389.0,NaN,NaN,NaN,NaN,NaN,N,E
4,0117,51,A,12.0,126.6,131.8,118.4,3.9,4.0,None,NaN,NaN,NaN,NaN,NaN,5.0,A,12.0,134.8,136.6,132.9,1.3,6,None,None,None,None,None,None,None,TROXLER,4640-B,509533,5389.0,NaN,NaN,NaN,NaN,NaN,N,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,0123,30,A,3.0,146.0,146.4,145.3,0.6,3.0,None,NaN,NaN,NaN,NaN,NaN,NaN,A,14.0,143.3,143.5,143.1,0.2,4,None,None,None,None,None,None,None,TROXLER,3440,20966,NaN,NaN,NaN,NaN,NaN,NaN,None,E
199,0119,30,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,A,3.0,144.6,146.1,143.4,1.4,4,None,None,None,None,None,None,None,TROXLER,3440,20966,NaN,NaN,NaN,NaN,NaN,NaN,None,E
200,0120,30,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,A,3.0,142.5,143.8,140.7,1.7,4,None,None,None,None,None,None,None,TROXLER,3440,20966,NaN,NaN,NaN,NaN,NaN,NaN,None,E
201,0122,30,A,3.0,141.8,143.5,140.0,1.8,3.0,None,NaN,NaN,NaN,NaN,NaN,NaN,A,3.0,144.5,146.2,142.3,2.0,4,None,None,None,None,None,None,None,TROXLER,3440,20966,NaN,NaN,NaN,NaN,NaN,NaN,None,E


In [270]:
df_7['AVERAGE_ATB'] = None
df_7['AVERAGE_BINDER'] = None
df_7['AVERAGE_SURFACE'] = None
df_7['AVERAGE_FRICTION'] = None
missing_sps1_pma_density_profile_counts = 0


for i in df_7.index:
# i=0
    data_sps1_pma_density_profile_sub = data_sps1_pma_density_profile[(data_sps1_pma_density_profile['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_sps1_pma_density_profile['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if len(data_sps1_pma_density_profile_sub) >0:
        df_7.loc[i,'AVERAGE_ATB'] = data_sps1_pma_density_profile_sub['AVERAGE_ATB'].values 
        df_7.loc[i,'AVERAGE_BINDER'] = data_sps1_pma_density_profile_sub['AVERAGE_BINDER'].values
        df_7.loc[i,'AVERAGE_SURFACE'] = data_sps1_pma_density_profile_sub['AVERAGE_SURFACE'].values 
        df_7.loc[i,'AVERAGE_FRICTION'] = data_sps1_pma_density_profile_sub['AVERAGE_FRICTION'].values 
        
    if len(data_sps1_pma_density_profile_sub) == 0:
        df_7['AVERAGE_ATB'] = 0
        df_7['AVERAGE_BINDER'] = 0
        df_7['AVERAGE_SURFACE'] = 0
        df_7['AVERAGE_FRICTION'] = 0
        
        if (df_7.loc[i,'PERM_ASPH_TREAT_BASE'] != 0) or (df_7.loc[i,'DENSE_GRD_ASPH_TREAT_BASE'] != 0) :
            df_7.loc[i,'AVERAGE_ATB'] = data_sps1_pma_density_profile['AVERAGE_ATB'].mean() 
            
        if (df_7.loc[i,'SURFACE_AND_BINDER'] != 0) :
            df_7.loc[i,'AVERAGE_BINDER'] = data_sps1_pma_density_profile['AVERAGE_BINDER'].mean()
            df_7.loc[i,'AVERAGE_SURFACE'] = data_sps1_pma_density_profile['AVERAGE_SURFACE'].mean()
            
        if (df_7.loc[i,'SURFACE_FRICTION'] != 0) :
            df_7.loc[i,'AVERAGE_FRICTION'] = data_sps1_pma_density_profile['AVERAGE_FRICTION'].mean()
            
        
        
#     df_7.loc[i,'AVERAGE_ATB'] = data_sps1_pma_density_profile_sub['AVERAGE_ATB'].values if len(data_sps1_pma_density_profile_sub) > 0 else 0 
#     df_7.loc[i,'AVERAGE_BINDER'] = data_sps1_pma_density_profile_sub['AVERAGE_BINDER'].values if len(data_sps1_pma_density_profile_sub) > 0 else 0 
#     df_7.loc[i,'AVERAGE_SURFACE'] = data_sps1_pma_density_profile_sub['AVERAGE_SURFACE'].values if len(data_sps1_pma_density_profile_sub) > 0 else 0 
#     df_7.loc[i,'AVERAGE_FRICTION'] = data_sps1_pma_density_profile_sub['AVERAGE_FRICTION'].values if len(data_sps1_pma_density_profile_sub) > 0 else 0  
    if len(data_sps1_pma_density_profile_sub) == 0 : missing_sps1_pma_density_profile_counts+=1


df_7['AVERAGE_ATB'] = df_7['AVERAGE_ATB'].fillna(0)
df_7['AVERAGE_BINDER'] = df_7['AVERAGE_BINDER'].fillna(0)
df_7['AVERAGE_SURFACE'] = df_7['AVERAGE_SURFACE'].fillna(0)
df_7['AVERAGE_FRICTION'] = df_7['AVERAGE_FRICTION'].fillna(0)


### pma_ac_properties

In [273]:
data_sps1_pma_ac_properties = pd.read_sql("select * from SPS1_PMA_AC_PROPERTIES", conn_sps)
data_sps1_pma_ac_properties

,SHRP_ID,STATE_CODE,LAYER_NO,ASPHALT_GRADE,ASPHALT_GRADE_OTHER,SOURCE,SOURCE_OTHER,ASPHALT_SPECIFIC_GRAVITY,ASPHALT_VISCOSITY_140,ASPHALT_VISCOSITY_275,PENETRATION_77,MODIFIER_1,MODIFIER_2,MODIFIER_1_OTHER,MODIFIER_2_OTHER,MODIFIER_QTY_1,MODIFIER_QTY_2,DUCTILITY_77,DUCTILITY_39,DUCTILITY_TEST_RATE,PENETRATION_39,RING_BALL_SOFT_PT,RECORD_STATUS
0,0113,51,4,4,None,77.0,"COASTAL FUEL, APEX, NC",1.025,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,E
1,0113,51,5,4,None,77.0,"COASTAL FUEL, APEX, NC",1.025,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,E
2,0114,51,4,4,None,77.0,"COASTAL FUEL, APEX, NC",1.025,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,E
3,0114,51,5,4,None,77.0,"COASTAL FUEL, APEX, NC",1.025,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,E
4,0115,51,3,4,None,77.0,"COASTAL FUEL, APEX, NC",1.025,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,None,None,None,None,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0123,30,4,17,PG 70-28,77.0,Montana Refining Company,NaN,NaN,NaN,NaN,13.0,None,None,None,NaN,NaN,NaN,None,None,None,None,E
484,0119,30,4,17,PG 70-28,77.0,Montana Refining Company,NaN,NaN,NaN,NaN,13.0,None,None,None,NaN,NaN,NaN,None,None,None,None,E
485,0120,30,4,17,PG 70-28,77.0,Montana Refining Company,NaN,NaN,NaN,NaN,13.0,None,None,None,NaN,NaN,NaN,None,None,None,None,E
486,0122,30,4,17,PG 70-28,77.0,Montana Refining Company,NaN,NaN,NaN,NaN,13.0,None,None,None,NaN,NaN,NaN,None,None,None,None,E


In [320]:
df_7['ASPHALT_SPECIFIC_GRAVITY'] = None
for i in df_7.index:
# i=0
    data_sps1_pma_ac_properties_sub = data_sps1_pma_ac_properties[(data_sps1_pma_ac_properties['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_sps1_pma_ac_properties['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_sps1_pma_ac_properties_sub['ASPHALT_SPECIFIC_GRAVITY'].mean())=='nan':
        df_7.loc[i,'ASPHALT_SPECIFIC_GRAVITY'] = data_sps1_pma_ac_properties['ASPHALT_SPECIFIC_GRAVITY'].mean()
    else:
        df_7.loc[i,'ASPHALT_SPECIFIC_GRAVITY'] = data_sps1_pma_ac_properties_sub['ASPHALT_SPECIFIC_GRAVITY'].mean()



### pma_aggregate_properties

In [242]:
data_sps1_pma_aggregate_properties = pd.read_sql("select * from SPS1_PMA_AGGREGATE_PROP", conn_sps)
data_sps1_pma_aggregate_properties

,SHRP_ID,STATE_CODE,LAYER_NO,AGGR_COMP_TYPE_C1,AGGR_COMP_TYPE_C2,AGGR_COMP_TYPE_C3,AGGR_COMP_TYPE_C1_OTHER,AGGR_COMP_TYPE_C2_OTHER,AGGR_COMP_TYPE_C3_OTHER,AGGR_COMP_PERCENT_C1,AGGR_COMP_PERCENT_C2,AGGR_COMP_PERCENT_C3,AGGR_COMP_TYPE_F1,AGGR_COMP_TYPE_F2,AGGR_COMP_TYPE_F3,AGGR_COMP_TYPE_F1_OTHER,AGGR_COMP_TYPE_F2_OTHER,AGGR_COMP_TYPE_F3_OTHER,AGGR_COMP_PERCENT_F1,AGGR_COMP_PERCENT_F2,AGGR_COMP_PERCENT_F3,MINERAL_FILLER,MINERAL_FILLER_OTHER,COARSE_AGGR_BULK_SPEC,FINE_AGGR_BULK_SPEC,MINERAL_FILLER_BULK_SPEC,AGGR_COMB_BULK_SPEC,EFFECTIVE_SPEC_GRAVITY,AGGR_DUR_TYPE_C1,AGGR_DUR_TYPE_C2,AGGR_DUR_TYPE_C3,AGGR_DUR_TYPE_M1,AGGR_DUR_RESULT_C1,AGGR_DUR_RESULT_C2,AGGR_DUR_RESULT_C3,AGGR_DUR_RESULT_M1,POLISH_VALUE,RECORD_STATUS
0,0115,5,2,1,NaN,NaN,None,None,None,100,NaN,NaN,1.0,NaN,NaN,None,None,None,100.0,NaN,NaN,NaN,None,NaN,NaN,NaN,2.723,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
1,0116,5,4,3,1.0,NaN,None,None,None,51,49.0,NaN,1.0,2.0,NaN,None,None,None,64.0,36.0,NaN,NaN,None,NaN,NaN,NaN,2.665,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
2,0116,5,3,1,3.0,NaN,None,None,None,74,26.0,NaN,1.0,NaN,NaN,None,None,None,100.0,NaN,NaN,NaN,None,NaN,NaN,NaN,2.705,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
3,0116,5,2,1,NaN,NaN,None,None,None,100,NaN,NaN,1.0,NaN,NaN,None,None,None,100.0,NaN,NaN,NaN,None,NaN,NaN,NaN,2.723,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
4,0117,5,5,3,1.0,NaN,None,None,None,51,49.0,NaN,1.0,2.0,NaN,None,None,None,64.0,36.0,NaN,NaN,None,NaN,NaN,NaN,2.665,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,0113,5,3,1,3.0,NaN,None,None,None,74,26.0,NaN,1.0,NaN,NaN,None,None,None,100.0,NaN,NaN,NaN,None,NaN,NaN,NaN,2.705,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
455,0114,5,4,3,1.0,NaN,None,None,None,51,49.0,NaN,1.0,2.0,NaN,None,None,None,64.0,36.0,NaN,NaN,None,NaN,NaN,NaN,2.665,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
456,0114,5,3,1,3.0,NaN,None,None,None,74,26.0,NaN,1.0,NaN,NaN,None,None,None,100.0,NaN,NaN,NaN,None,NaN,NaN,NaN,2.705,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E
457,0115,5,4,3,2.0,NaN,None,None,None,51,49.0,NaN,1.0,2.0,NaN,None,None,None,64.0,36.0,NaN,NaN,None,NaN,NaN,NaN,2.665,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN,E


In [329]:
df_7['AGGR_COMP_PERCENT_C1'] = None
for i in df_7.index:
# i=0
    data_sps1_pma_aggregate_properties_sub = data_sps1_pma_aggregate_properties[(data_sps1_pma_aggregate_properties['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_sps1_pma_aggregate_properties['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_sps1_pma_aggregate_properties_sub['AGGR_COMP_PERCENT_C1'].mean())=='nan':
        df_7.loc[i,'AGGR_COMP_PERCENT_C1'] = data_sps1_pma_aggregate_properties['AGGR_COMP_PERCENT_C1'].mean()
    else:
        df_7.loc[i,'AGGR_COMP_PERCENT_C1'] = data_sps1_pma_aggregate_properties_sub['AGGR_COMP_PERCENT_C1'].mean()



In [330]:

# data_sps1_pma_aggregate_properties[(data_sps1_pma_aggregate_properties['STATE_CODE'] == 1)&(data_sps1_pma_aggregate_properties['SHRP_ID'] == '0102')]

### pma_mixture_prop

In [324]:
data_sps1_pma_mixture_prop = pd.read_sql("select * from SPS1_PMA_MIXTURE_PROP", conn_sps)
data_sps1_pma_mixture_prop

,SHRP_ID,STATE_CODE,LAYER_NO,SAMPLE_TYPE,MAX_SPEC_GRAVITY,BULK_SPEC_GRAVITY_MEAN,BULK_SPEC_GRAVITY_MIN,BULK_SPEC_GRAVITY_MAX,BULK_SPEC_GRAVITY_STD_DEV,NO_BULK_SPEC_GRAVITY,ASPHALT_CONTENT_MEAN,ASPHALT_CONTENT_MIN,ASPHALT_CONTENT_MAX,ASPHALT_CONTENT_STD_DEV,NO_SAMP_ASPHALT_CONTENT,PCT_AIR_VOIDS_MEAN,PCT_AIR_VOIDS_MIN,PCT_AIR_VOIDS_MAX,PCT_AIR_VOIDS_STD_DEV,NO_SAMP_PCT_AIR_VOIDS,VOIDS_MINERAL_AGGR,EFF_ASPHALT_CONTENT,MARSHALL_STABILITY,NUMBER_OF_BLOWS,MARSHALL_FLOW,HVEEM_STABILITY,HVEEM_COHESIOMETER,ASPHALT_PLANT_TYPE,ASPHALT_PLANT_TYPE_OTHER,ANTISTRIP_AGENT_TYPE,ANTISTRIP_AGENT_TYPE_OTHER,ANTISTRIP_AGENT_CODE,ANTISTRIP_AGENT_AMOUNT,RECORD_STATUS
0,0112,12,5,1.0,2.340,2.230,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,1.0,4.7,NaN,NaN,NaN,1.0,15.4,5.0,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
1,0112,12,6,1.0,2.342,2.235,NaN,NaN,NaN,1.0,6.5,NaN,NaN,NaN,1.0,4.6,NaN,NaN,NaN,1.0,16.2,5.4,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
2,0161,12,3,1.0,2.340,2.230,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,1.0,4.7,NaN,NaN,NaN,1.0,15.4,5.0,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
3,0161,12,4,1.0,2.342,2.235,NaN,NaN,NaN,1.0,6.5,NaN,NaN,NaN,1.0,4.6,NaN,NaN,NaN,1.0,16.2,5.4,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
4,0110,35,6,1.0,2.377,2.282,NaN,NaN,NaN,1.0,4.5,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN,1.0,13.9,4.3,2114.0,75.0,9.0,NaN,None,2.0,None,NaN,None,NaN,NaN,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,0111,12,4,1.0,2.311,2.233,NaN,NaN,NaN,1.0,6.5,NaN,NaN,NaN,1.0,3.4,NaN,NaN,NaN,1.0,15.9,5.8,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
429,0111,12,5,1.0,2.340,2.230,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,1.0,4.7,NaN,NaN,NaN,1.0,15.4,5.0,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
430,0111,12,6,1.0,2.342,2.235,NaN,NaN,NaN,1.0,6.5,NaN,NaN,NaN,1.0,4.6,NaN,NaN,NaN,1.0,16.2,5.4,NaN,75.0,NaN,NaN,None,2.0,None,NaN,None,NaN,NaN,E
431,0112,12,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,NaN,NaN,E


In [ ]:
df_7['ASPHALT_CONTENT_MEAN'] = None
df_7['PCT_AIR_VOIDS_MEAN'] = None
for i in df_7.index:
# i=0
    data_sps1_pma_mixture_prop_sub = data_sps1_pma_mixture_prop[(data_sps1_pma_mixture_prop['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_sps1_pma_mixture_prop['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_sps1_pma_mixture_prop_sub['ASPHALT_CONTENT_MEAN'].mean())=='nan':
        df_7.loc[i,'ASPHALT_CONTENT_MEAN'] = data_sps1_pma_mixture_prop['ASPHALT_CONTENT_MEAN'].mean()
    else:
        df_7.loc[i,'ASPHALT_CONTENT_MEAN'] = data_sps1_pma_mixture_prop_sub['ASPHALT_CONTENT_MEAN'].mean()
        
    if str(data_sps1_pma_mixture_prop_sub['PCT_AIR_VOIDS_MEAN'].mean())=='nan':
        df_7.loc[i,'PCT_AIR_VOIDS_MEAN'] = data_sps1_pma_mixture_prop['PCT_AIR_VOIDS_MEAN'].mean()
    else:
        df_7.loc[i,'PCT_AIR_VOIDS_MEAN'] = data_sps1_pma_mixture_prop_sub['PCT_AIR_VOIDS_MEAN'].mean()



In [333]:

# data_sps1_pma_mixture_prop[(data_sps1_pma_mixture_prop['STATE_CODE'] == 1)&(data_sps1_pma_mixture_prop['SHRP_ID'] == '0102')]

### unbound_layer_prop_exp

In [326]:
data_unbound_layer_prop_exp = pd.read_sql("select * from UNBOUND_LAYER_PROP_EXP", conn_consolidated)
data_unbound_layer_prop_exp

,UNBOUND_LAYER_PROP_ID,STATE_CODE,SHRP_ID,LAYER_NO,AASHTO_SOIL_CLASS,PLASTICITY_INDEX,LIQUID_LIMIT,PLASTIC_LIMIT,MAX_LAB_DRY_DENSITY,OPTIMUM_LAB_MOISTURE,MAX_DRY_DENSITY_TEST,MAX_DRY_DENSITY_TEST_OTHER,COMPACTIVE_ENERGY,IN_SITU_DRY_DEN_MEAN,IN_SITU_DRY_DEN_MIN,IN_SITU_DRY_DEN_MAX,NO_SAMP_IN_SITU_DRY_DEN,IN_SITU_DRY_DEN_STD_DEV,IN_SITU_MOISTURE_MEAN,IN_SITU_MOISTURE_MIN,IN_SITU_MOISTURE_MAX,NO_SAMP_IN_SITU_MOISTURE,IN_SITU_MOISTURE_STD_DEV,COMP_STRENGTH_MEAN,COMP_STRENGTH_MIN,COMP_STRENGTH_MAX,NO_COMP_STRENGTH_TESTS,COMP_STRENGTH_STD_DEV,COMP_TEST_TYPE,COMP_TEST_TYPE_OTHER,CONFINING_PRESSURE,CALCIUM_CARBONATE_CONTENT,CALIFORNIA_BEARING_RATIO,RESISTANCE,MODULUS_UNBOUND_REACTION,TEST_TYPE,RECORD_STATUS,MODULUS_SUBGRADE_REACTION
0,1,1,0502,2,4,NaN,NaN,NaN,124.0,10.5,2,None,NaN,118.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
1,2,1,0502,3,1,NaN,NaN,NaN,136.0,5.8,2,None,NaN,136.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
2,3,1,0503,2,4,NaN,NaN,NaN,124.0,10.5,2,None,NaN,118.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
3,4,1,0503,3,1,NaN,NaN,NaN,136.0,5.8,2,None,NaN,136.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
4,5,1,0504,2,4,NaN,NaN,NaN,124.0,10.5,2,None,NaN,118.0,NaN,NaN,NaN,NaN,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,1801,88,1646,2,1,NaN,NaN,NaN,125.0,7.5,1,None,NaN,124.0,119.0,131.0,13.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,5.0,NaN,None,None,E,None
1832,1802,88,1647,2,1,NaN,NaN,NaN,131.0,5.7,1,None,NaN,132.0,122.0,142.0,49.0,4.0,5.0,NaN,NaN,32.0,1.0,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None
1833,1803,88,1647,3,1,NaN,NaN,NaN,117.0,11.8,1,None,NaN,116.0,111.0,123.0,6.0,5.0,10.0,9.0,12.0,6.0,1.0,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,3.0,NaN,None,None,E,None
1834,1804,89,1021,2,4,NaN,NaN,NaN,104.0,10.6,None,None,NaN,NaN,NaN,NaN,NaN,NaN,7.0,5.0,10.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,E,None


In [355]:
df_7['UNBOUND_PLASTICITY_INDEX'] = None
for i in df_7.index:
# i=0
    data_unbound_layer_prop_exp_sub = data_unbound_layer_prop_exp[(data_unbound_layer_prop_exp['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_unbound_layer_prop_exp['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_unbound_layer_prop_exp_sub['PLASTICITY_INDEX'].mean())=='nan':
        df_7.loc[i,'UNBOUND_PLASTICITY_INDEX'] = data_unbound_layer_prop_exp['PLASTICITY_INDEX'].mean()
    else:
        df_7.loc[i,'UNBOUND_PLASTICITY_INDEX'] = data_unbound_layer_prop_exp_sub['PLASTICITY_INDEX'].mean()


### SUBGRADE_LAYER_PROP_EXP

In [356]:
data_subgrade_layer_prop_exp = pd.read_sql("select * from SUBGRADE_LAYER_PROP_EXP", conn_consolidated)
data_subgrade_layer_prop_exp

,SUBGRADE_LAYER_PROP_ID,STATE_CODE,SHRP_ID,LAYER_NO,AASHTO_SOIL_CLASS,CALIFORNIA_BEARING_RATIO,RESISTANCE,SUBGRADE_REACTION_MODULUS,TEST_TYPE,PLASTICITY_INDEX,LIQUID_LIMIT,MAXIMUM_LAB_DRY_DENSITY,OPTIMUM_LAB_MOISTURE_CONTENT,MAX_LAB_DRY_DENSITY_TEST,MAX_LAB_DRY_DENSITY_TEST_OTHER,COMPACTIVE_ENERGY_OTHER,IN_SITU_DRY_DEN_MEAN,IN_SITU_DRY_DEN_MAX,IN_SITU_DRY_DEN_MIN,NO_SAMP_IN_SITU_DRY_DEN,IN_SITU_DRY_DEN_STD_DEV,IN_SITU_MOISTURE_OPT_MEAN,IN_SITU_MOISTURE_OPT_MIN,IN_SITU_MOISTURE_OPT_MAX,NO_SAMP_IN_SITU_MOISTURE_OPT,IN_SITU_MOISTURE_OPT_STD_DEV,IN_SITU_DRY_DEN_PCF_MEAN,IN_SITU_DRY_DEN_PCF_MIN,IN_SITU_DRY_DEN_PCF_MAX,NO_SAMP_IN_SITU_DRY_DEN_PCF,IN_SITU_DRY_DEN_PCF_STD_DEV,IN_SITU_MOISTURE_DW_MEAN,IN_SITU_MOISTURE_DW_MIN,IN_SITU_MOISTURE_DW_MAX,NO_SAMP_IN_SITU_MOISTURE_DW,IN_SITU_MOISTURE_DW_STD_DEV,LAB_DENSITY_MIN,LAB_DENSITY_MAX,RELATIVE_DENSITY_MEAN,RELATIVE_DENSITY_MIN,NO_RELATIVE_DENSITY_TESTS,RELATIVE_DENSITY_MAX,RELATIVE_DENSITY_STD_DEV,SOIL_SUCTION,EXPANSION_INDEX,SWELL_PRESSURE_VALUE,SWELL_PRESSURE_CODE,SWELL_PRESSURE_CODE_OTHER,PERCENT_FINER_02,AVG_RATE_HEAVE,FROST_SUSCEPTIBILITY_CODE,RECORD_STATUS
0,4184,1,0502,1,8,33.0,NaN,NaN,None,7.0,25.0,127.0,10.3,1,None,NaN,98.0,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,None,E
1,4185,1,0503,1,8,33.0,NaN,NaN,None,7.0,25.0,127.0,10.3,1,None,NaN,98.0,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,None,E
2,4186,1,0504,1,8,33.0,NaN,NaN,None,7.0,25.0,127.0,10.3,1,None,NaN,98.0,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,None,E
3,4187,1,0505,1,8,33.0,NaN,NaN,None,7.0,25.0,127.0,10.3,1,None,NaN,98.0,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,None,E
4,4188,1,0506,1,8,33.0,NaN,NaN,None,7.0,25.0,127.0,10.3,1,None,NaN,98.0,NaN,NaN,NaN,NaN,97.0,NaN,NaN,NaN,NaN,124.0,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,None,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,5562,90,B330,1,7,NaN,NaN,NaN,None,NaN,NaN,126.0,10.6,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,4,E
1379,5563,90,B331,1,7,NaN,NaN,NaN,None,NaN,NaN,126.0,10.6,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,4,E
1380,5564,90,B340,1,7,NaN,NaN,NaN,None,NaN,NaN,126.0,10.6,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,4,E
1381,5565,90,B350,1,7,NaN,NaN,NaN,None,NaN,NaN,126.0,10.6,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,4,E


In [358]:
df_7['SUBGRADE_PLASTICITY_INDEX'] = None
for i in df_7.index:
# i=0
    data_subgrade_layer_prop_exp_sub = data_subgrade_layer_prop_exp[(data_unbound_layer_prop_exp['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_unbound_layer_prop_exp['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_subgrade_layer_prop_exp_sub['PLASTICITY_INDEX'].mean())=='nan':
        df_7.loc[i,'SUBGRADE_PLASTICITY_INDEX'] = data_subgrade_layer_prop_exp['PLASTICITY_INDEX'].mean()
    else:
        df_7.loc[i,'SUBGRADE_PLASTICITY_INDEX'] = data_subgrade_layer_prop_exp_sub['PLASTICITY_INDEX'].mean()


### AC#200

In [360]:
data_ac_aggr_gradation_exp = pd.read_sql("select * from AC_AGGR_GRADATION_EXP", conn_consolidated)
data_ac_aggr_gradation_exp

,AC_AGGR_GRADATION_ID,STATE_CODE,SHRP_ID,LAYER_NO,AGGR_TYPE,TWO,ONE_AND_HALF,ONE,SEVEN_EIGHTHS,THREE_FOURTHS,FIVE_EIGHTHS,ONE_HALF,THREE_EIGHTHS,NO_4,NO_8,NO_10,NO_16,NO_30,NO_40,NO_50,NO_80,NO_100,NO_200,RECORD_STATUS
0,16074,1,0502,4,4,NaN,100.0,NaN,NaN,90.0,NaN,72.0,63.0,46.0,38.0,NaN,NaN,30.0,NaN,16.0,NaN,3.0,1.0,E
1,16075,1,0502,5,4,NaN,NaN,NaN,NaN,NaN,NaN,100.0,87.0,62.0,54.0,NaN,NaN,39.0,NaN,23.0,NaN,11.0,6.0,E
2,16076,1,0503,4,4,NaN,100.0,NaN,NaN,90.0,NaN,72.0,63.0,46.0,38.0,NaN,NaN,30.0,NaN,16.0,NaN,3.0,1.0,E
3,16077,1,0503,5,4,NaN,NaN,NaN,NaN,NaN,NaN,100.0,87.0,62.0,54.0,NaN,NaN,39.0,NaN,23.0,NaN,11.0,6.0,E
4,16078,1,0504,4,4,NaN,100.0,NaN,NaN,90.0,NaN,72.0,63.0,46.0,38.0,NaN,NaN,30.0,NaN,16.0,NaN,3.0,1.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206,18579,48,3629,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,64.0,NaN,39.0,NaN,NaN,23.0,NaN,9.0,4.0,4.0,E
3207,18580,48,3629,5,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.0,64.0,NaN,39.0,NaN,NaN,23.0,NaN,9.0,4.0,4.0,E
3208,18581,48,3669,4,4,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,75.0,NaN,56.0,NaN,NaN,47.0,NaN,18.0,NaN,6.0,E
3209,18582,48,3669,5,4,NaN,NaN,NaN,NaN,NaN,NaN,100.0,99.0,72.0,NaN,56.0,NaN,NaN,47.0,NaN,21.0,NaN,6.0,E


In [377]:
df_7['AC_NO_200'] = None
x =0
for i in df_7.index:
# i=0
    data_ac_aggr_gradation_exp_sub = data_ac_aggr_gradation_exp[(data_ac_aggr_gradation_exp['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_ac_aggr_gradation_exp['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_ac_aggr_gradation_exp_sub['NO_200'].mean())=='nan':
        x =x+1
        df_7.loc[i,'AC_NO_200'] = data_ac_aggr_gradation_exp['NO_200'].mean()
    else:
        df_7.loc[i,'AC_NO_200'] = data_ac_aggr_gradation_exp_sub['NO_200'].mean()


### UNBOUND#200

In [155]:
data_unbound_gradation_exp = pd.read_sql("select * from UNBOUND_GRADATION_EXP", conn_consolidated)
data_unbound_gradation_exp

,UNBOUND_GRADATION_ID,STATE_CODE,SHRP_ID,LAYER_NO,GRADATION_TYPE,TWO,ONE_AND_HALF,ONE,SEVEN_EIGHTHS,THREE_FOURTHS,FIVE_EIGHTHS,ONE_HALF,THREE_EIGHTHS,NO_4,NO_8,NO_10,NO_16,NO_30,NO_40,NO_50,NO_80,NO_100,NO_200,RECORD_STATUS
0,1,1,1021,2,3,NaN,100.0,91.0,NaN,NaN,NaN,NaN,NaN,47.0,37.0,NaN,NaN,NaN,32.0,NaN,NaN,NaN,16.0,E
1,2,1,4126,2,3,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,58.0,NaN,22.0,NaN,NaN,12.0,NaN,NaN,NaN,6.0,E
2,3,1,4084,2,3,NaN,100.0,94.0,NaN,NaN,NaN,NaN,NaN,64.0,NaN,47.0,NaN,NaN,44.0,NaN,NaN,NaN,14.0,E
3,4,1,4084,3,3,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN,15.0,NaN,NaN,7.0,NaN,NaN,NaN,2.0,E
4,5,1,4073,3,3,NaN,100.0,92.0,NaN,NaN,NaN,79.0,NaN,59.0,42.0,NaN,27.0,NaN,17.0,15.0,NaN,NaN,10.0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,859,48,1178,3,3,NaN,100.0,97.0,80.0,NaN,70.0,NaN,56.0,43.0,NaN,33.0,NaN,NaN,24.0,NaN,NaN,NaN,NaN,E
1344,860,35,6401,3,3,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,30.0,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,E
1345,861,35,1003,2,3,NaN,NaN,NaN,100.0,97.0,NaN,NaN,NaN,35.0,23.0,NaN,NaN,NaN,13.0,NaN,NaN,NaN,7.0,E
1346,862,48,L350,2,3,NaN,NaN,NaN,86.0,NaN,70.0,NaN,54.0,40.0,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E


In [156]:
df_7['UNBOUND_NO_200'] = None
x= 0
for i in df_7.index:
# i=0
    data_unbound_gradation_exp_sub = data_unbound_gradation_exp[(data_unbound_gradation_exp['STATE_CODE'] == df_7.loc[i,'STATE_CODE']) & \
                                                                    (data_unbound_gradation_exp['SHRP_ID'] == df_7.loc[i,'SHRP_ID'])]
    if str(data_unbound_gradation_exp_sub['NO_200'].mean())=='nan':
        x=x+1
        df_7.loc[i,'UNBOUND_NO_200'] = data_unbound_gradation_exp['NO_200'].mean()
    else:
        df_7.loc[i,'UNBOUND_NO_200'] = data_unbound_gradation_exp_sub['NO_200'].mean()


In [379]:
df_7

,STATE_CODE,SHRP_ID,CONSTRUCTION_NO,DATE,ASSIGN_DATE,MEAN_IRI_LEFT_WHEEL_PATH,MEAN_IRI_RIGHT_WHEEL_PATH,MEAN_MRI,GATOR_CRACK,BLOCK_CRACK,EDGE_CRACK,LONGITUDINAL_CRACK_WP,LONGITUDINAL_CRACK_NWP,TRANS_CRACK_NUMBER,TRANS_CRACK_LENGTH,PATCH_NUMBER,PATCH_AREA,POTHOLES_NUMBER,POTHOLES_AREA,SHOVING_NO,SHOVING_A,BLEEDING,POLISH_AGG_A,RAVELING,PUMPING_NO,PUMPING_L,WP_LENGTH_CRACKED,TRANS_CRACK_L_GT183,RUTTING,FRICTION,HYDRAULIC_CONDUCTIVITY,BC_LAYER_NO_1_MODULUS,BC_LAYER_NO_2_MODULUS,BC_LAYER_NO_3_MODULUS,BC_LAYER_NO_4_MODULUS,BC_LAYER_NO_5_MODULUS,AGE,INITIAL_IRI,DENSE_GRADE_AGG_BASE,PERM_ASPH_TREAT_BASE,DENSE_GRD_ASPH_TREAT_BASE,SURFACE_AND_BINDER,SURFACE_FRICTION,MAX_MON_HUM_AVG,MIN_MON_HUM_AVG,TOTAL_MON_PRECIP,MEAN_MON_SOLAR_AVG,MAX_MON_TEMP_AVG,MIN_MON_TEMP_AVG,DAYS_ABOVE_32_C,DAYS_BELOW_0_C,FREEZE_INDEX,MEAN_MON_WIND_AVG,KESAL_YEAR,AVERAGE_ATB,AVERAGE_BINDER,AVERAGE_SURFACE,AVERAGE_FRICTION,ASPHALT_SPECIFIC_GRAVITY,AGGR_COMP_PERCENT_C1,UNBOUND_PLASTICITY_INDEX,SUBGRADE_PLASTICITY_INDEX,AC_NO_200,UNBOUND_NO_200
0,1,0101,1.0,1995-10-30,1991-04-30,0.6572,0.6574,0.6572,-3.932266,0.0,0.0,12.365742,0.173128,-3.523890,-1.443286,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,20.049527,-0.047778,0.190046,-0.248165,-0.149438,6.435719,37.814286,9945.685558,1195.443398,519.017952,127.704923,52.343468,71.516652,4.501119,0.635183,7.916364,0.000000,0.000000,6.830909,0.00,95.4143,40.2,59.7143,226.029,29.0571,16.0571,9.42857,0.428571,0,1.4,547.404,0.000000,0.000000,0.000000,0.0,1.02255,100,6.90855,14.3457,4.92646,8.54327
1,1,0101,1.0,1996-01-10,1991-04-30,0.6962,0.6384,0.6670,-2.481933,0.0,0.0,12.247009,0.170197,-2.813874,-1.158582,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.864897,-0.028871,0.240824,4.238165,-0.122338,6.466592,36.785714,9945.685558,1195.443398,519.017952,127.704923,52.343468,71.516652,4.698248,0.635183,7.916364,0.000000,0.000000,6.830909,0.00,95.71,42.12,95.8,194.92,25.73,12.84,6.6,2.6,0.7,1.42,547.404,0.000000,0.000000,0.000000,0.0,1.02255,100,6.90855,14.3457,4.92646,8.54327
2,1,0101,1.0,1996-04-08,1991-04-30,0.6508,0.7148,0.6828,-0.689160,0.0,0.0,12.100242,0.166574,-1.936216,-0.806657,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.636674,-0.005501,0.303591,9.783768,-0.088840,6.504755,35.514286,9945.685558,1195.443398,519.017952,127.704923,52.343468,71.516652,4.941922,0.635183,7.916364,0.000000,0.000000,6.830909,0.00,95.2538,42.3846,111.077,181.615,23.2923,10.3385,5.07692,5,3.23077,1.58462,547.404,0.000000,0.000000,0.000000,0.0,1.02255,100,6.90855,14.3457,4.92646,8.54327
3,1,0101,1.0,1996-08-12,1991-04-30,0.6804,0.7132,0.6970,1.848923,0.0,0.0,11.892460,0.161444,-0.693688,-0.308425,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.313571,0.027586,0.392453,17.634846,-0.041416,6.558783,33.714286,9945.685558,1195.443398,519.017952,127.704923,52.343468,71.516652,5.286898,0.635183,7.916364,0.000000,0.000000,6.830909,0.00,95.8235,42.1765,103.824,198.318,24.6706,11.6529,6.11765,3.88235,2.47059,1.55294,547.404,0.000000,0.000000,0.000000,0.0,1.02255,100,6.90855,14.3457,4.92646,8.54327
4,1,0101,1.0,1996-10-11,1991-04-30,0.6836,0.6724,0.6780,3.057534,0.0,0.0,11.793516,0.159002,-0.102008,-0.071172,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,19.159712,0.043341,0.434768,21.373454,-0.018832,6.584510,32.857143,9945.685558,1195.443398,519.017952,127.704923,52.343468,71.516652,5.451173,0.635183,7.916364,0.000000,0.000000,6.830909,0.00,96.2526,43.2632,101.842,198.179,25.1842,12.3632,6.10526,3.47368,2.21053,1.5,547.404,0.000000,0.000000,0.000000,0.0,1.02255,100,6.90855,14.3457,4.92646,8.54327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,55,0124,1.0,2002-05-10,1997-01-01,0.9748,1.1158,1.0454,0.000000,0.0,0.0,0.000000,168.834606,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,102.013172,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,8.205664,46.921053,4868.161670,1195.443398,

In [7]:
df_7.to_csv('data_iri.csv',index = False)
# df_7 = pd.read_csv('data_iri.csv', dtype={'SHRP_ID': str},index_col=None)